In [28]:
import json
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import datetime
from tqdm import tqdm

### generation

In [29]:
def load_and_preprocess_data(meta_data_path, data_dir):
    """
    Load and preprocess all solar plant data files with forecast data
    
    Args:
        meta_data_path: Path to metadata CSV
        data_dir: Directory containing parquet files with forecast data
    
    Returns:
        processed_data: Dictionary of preprocessed dataframes by plant
        feature_cols: List of feature columns (excluding forecast columns)
        forecast_cols_14: List of 14시 forecast columns
        forecast_cols_20: List of 20시 forecast columns
        target_col: Target column name
        plants_info: Dictionary with information about each plant
        meta_data: DataFrame containing plant metadata
    """
    # Load metadata
    meta_data = pd.read_csv(meta_data_path)
    
    # Get all files
    files = os.listdir(data_dir)
    
    # Initialize dictionaries to store processed data
    processed_data = {}
    plants_info = {}
    
    # Define column types
    time_cols = ['date', 'time', 'sunrise', 'sunset']
    
    # Weather/environment columns (numerical)
    weather_numeric_cols = [
        'temperature', 'humidity', 'ws', 'pv', 'pa', 'ps', 
        'ss', 'lcsCh','dc10Tca', 'dc10LmcsCa', 'vs', 'ts'
    ]
    
    # Circular features (need sin/cos transformation)
    circular_cols = ['wd']
    
    # Weather columns with special imputation (zero)
    weather_zero_impute_cols = ['rn', 'icsr']
    
    # Categorical weather columns
    categorical_cols = ['미세먼지', '초미세먼지']  # lcsCh 제외
    
    # Air quality columns (numerical)
    air_numeric_cols = ['SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25']
    
    # Solar plant output columns 
    plant_output_cols = ['총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별_kw)', '최소(시간별_kw)', 'value']
    
    # 14시 예보 컬럼 (직접 정의)
    forecast_cols_14 = ['temp_14', 'wd_14', 'sc_14', 'ws_14', 'pp_14']
    
    # 20시 예보 컬럼 (직접 정의)
    forecast_cols_20 = ['temp_20', 'wd_20', 'sc_20', 'ws_20', 'pp_20']
    
    # 예보 컬럼과 그 파생 컬럼들 (나중에 제외할 용도)
    all_forecast_cols = forecast_cols_14 + forecast_cols_20 + [
        'wd_14_sin', 'wd_14_cos', 'wd_20_sin', 'wd_20_cos'
    ]
    
    # Target column
    target_col = 'value'
    
    # All numeric columns for imputation
    all_numeric_cols = (weather_numeric_cols + air_numeric_cols + plant_output_cols +
                       [col for col in forecast_cols_14 + forecast_cols_20 
                        if col not in ['sc_14', 'sc_20']])  # 하늘상태(sc)는 범주형
    
    # 샘플 파일 로드하여 실제 컬럼 확인
    sample_files = [f for f in files if f.endswith('.parquet')]
    if sample_files:
        sample_file = sample_files[0]
        try:
            sample_df = pd.read_parquet(os.path.join(data_dir, sample_file))
            # 실제 존재하는 예보 컬럼만 필터링
            forecast_cols_14 = [col for col in forecast_cols_14 if col in sample_df.columns]
            forecast_cols_20 = [col for col in forecast_cols_20 if col in sample_df.columns]
            print(f"Available 14시 forecast columns: {forecast_cols_14}")
            print(f"Available 20시 forecast columns: {forecast_cols_20}")
        except Exception as e:
            print(f"Warning: Could not check columns from sample file: {e}")
    
    # Analyze all files to collect plant information
    print("Collecting plant information...")
    for file_name in tqdm(files):
        if not file_name.endswith('.parquet'):
            continue
            
        plant_id = file_name.split('.')[0]  # Assuming filename contains plant ID
        file_path = os.path.join(data_dir, file_name)
        
        # Load data - exclude '호기' column from the beginning
        df = pd.read_parquet(file_path)
        if '호기' in df.columns:
            df = df.drop(columns=['호기'])
        
        # Extract date range information
        if 'date' in df.columns:
            min_date = df['date'].min()
            max_date = df['date'].max()
            date_range = f"{min_date} ~ {max_date}"
        else:
            date_range = "Unknown"
        
        # Store plant information
        plants_info[plant_id] = {
            'file_name': file_name,
            'date_range': date_range,
            'data_length': len(df),
            'columns': list(df.columns)
        }
    
    print(f"Found {len(plants_info)} plants")
    
    # Process each file
    print("Processing each plant's data...")
    for file_name in tqdm(files):
        if not file_name.endswith('.parquet'):
            continue
            
        plant_id = file_name.split('.')[0]
        file_path = os.path.join(data_dir, file_name)
        
        # Load data - exclude '호기' column
        df = pd.read_parquet(file_path)
        if '호기' in df.columns:
            df = df.drop(columns=['호기'])
        
        # Process datetime columns - 시간 관련 특성 생성
        df = process_time_features(df, time_cols)
        
        # Process special weather features (wd, etc.)
        df = process_weather_features(df, circular_cols)
        
        # Process circular forecast features (wd_14, wd_20)
        df = process_forecast_features(df, ['wd_14', 'wd_20'])

        # NaN 값 처리 전 상태 출력 (NaN이 있는 컬럼만)
        print("===== Missing Values Before Processing =====")
        for col in df.columns:
            nan_count = df[col].isna().sum()
            if nan_count > 0:
                nan_ratio = df[col].isna().mean() * 100
                print(f"{col}: {nan_count}개 ({nan_ratio:.1f}%)")

        # Handle missing values according to the provided strategy
        df = handle_missing_values(
            df, 
            numeric_cols=all_numeric_cols,
            zero_impute_cols=weather_zero_impute_cols,
            categorical_cols=categorical_cols + ['sc_14', 'sc_20'],
            circular_cols=['wd', 'wd_sin', 'wd_cos', 'wd_14_sin', 'wd_14_cos', 
                          'wd_20_sin', 'wd_20_cos']
        )
        
        # NaN 값 처리 후 상태 출력 (NaN이 있는 컬럼만)
        print("\n===== Missing Values After Processing =====")
        nan_exists = False

        for col in df.columns:
            nan_count = df[col].isna().sum()
            if nan_count > 0:
                nan_ratio = df[col].isna().mean() * 100
                print(f"{col}: {nan_count}개 ({nan_ratio:.1f}%)")
                nan_exists = True

        if not nan_exists:
            print("모든 결측치가 성공적으로 처리되었습니다. NaN 값이 없습니다.")
                        
        # Save to dictionary
        processed_data[plant_id] = df
    
    # Define feature columns (excluding forecast columns and target)
    if len(processed_data) > 0:
        sample_df = processed_data[list(processed_data.keys())[0]]
        
        # Target column is the hourly generation ('value')
        target_col = 'value'
        
        # Excluded columns (raw time columns, derived intermediate columns, and forecast columns)
        excluded_cols = time_cols + ['datetime', 'hour', 'day_of_year', 'month', 'date_only'] + all_forecast_cols
        
        # Include 'value' in feature columns for historical data (과거 발전량 포함)
        feature_cols = [col for col in sample_df.columns if col not in excluded_cols]
        
        print(f"Base feature columns (excluding forecast): {len(feature_cols)}")
        print(f"14시 forecast columns: {len(forecast_cols_14)}")
        print(f"20시 forecast columns: {len(forecast_cols_20)}")
    else:
        feature_cols = []
    
    return processed_data, feature_cols, forecast_cols_14, forecast_cols_20, target_col, plants_info, meta_data

def process_time_features(df, time_cols):
    """Process time-related columns and extract useful features"""
    
    # Convert date and time to datetime if they exist
    if 'date' in df.columns and 'time' in df.columns:
        # Ensure date and time are string type
        df['date'] = df['date'].astype(str)
        df['time'] = df['time'].astype(str)
        
        # Create datetime column
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], errors='coerce')
        df['hour'] = df['datetime'].dt.hour
        
        # Create cyclical time features
        df['time_hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
        df['time_hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
        
        # Day of year (for seasonal patterns)
        df['day_of_year'] = df['datetime'].dt.dayofyear
        df['time_day_sin'] = np.sin(2 * np.pi * df['day_of_year']/365)
        df['time_day_cos'] = np.cos(2 * np.pi * df['day_of_year']/365)
        
        # Month as a cyclical feature (for seasonal patterns)
        df['month'] = df['datetime'].dt.month
        df['time_month_sin'] = np.sin(2 * np.pi * df['month']/12)
        df['time_month_cos'] = np.cos(2 * np.pi * df['month']/12)
        
        # Create features for day/night based on sunrise/sunset
        if 'sunrise' in df.columns and 'sunset' in df.columns:
            try:
                # Convert sunrise/sunset to datetime
                df['sunrise'] = pd.to_datetime(df['date'] + ' ' + df['sunrise'], errors='coerce')
                df['sunset'] = pd.to_datetime(df['date'] + ' ' + df['sunset'], errors='coerce')
                
                # Is daylight (1 if current time is between sunrise and sunset)
                df['time_is_daylight'] = ((df['datetime'] >= df['sunrise']) & 
                                     (df['datetime'] <= df['sunset'])).astype(int)
                
                # Hours since sunrise and hours until sunset
                df['time_hours_since_sunrise'] = (df['datetime'] - df['sunrise']).dt.total_seconds() / 3600
                df['time_hours_until_sunset'] = (df['sunset'] - df['datetime']).dt.total_seconds() / 3600
                
                # Replace negative values with 0 (before sunrise or after sunset)
                df['time_hours_since_sunrise'] = df['time_hours_since_sunrise'].clip(lower=0)
                df['time_hours_until_sunset'] = df['time_hours_until_sunset'].clip(lower=0)
            except Exception as e:
                print(f"Error processing sunrise/sunset: {e}")
    
    return df

def process_weather_features(df, circular_cols):
    """특수한 날씨 피처들에 대한 전처리"""
    
    # 풍향(wd)의 순환적 특성 처리
    if 'wd' in df.columns and 'wd' in circular_cols:
        # 풍향이 0-360도 범위인지 확인
        max_wd = df['wd'].max()
        if pd.notna(max_wd):  # NaN 체크
            if max_wd <= 360:
                # 사인/코사인 변환
                df['wd_sin'] = np.sin(2 * np.pi * df['wd'] / 360)
                df['wd_cos'] = np.cos(2 * np.pi * df['wd'] / 360)
            # 만약 다른 범위라면 적절히 조정
            else:
                df['wd_sin'] = np.sin(2 * np.pi * df['wd'] / max_wd)
                df['wd_cos'] = np.cos(2 * np.pi * df['wd'] / max_wd)
    
    return df

def process_forecast_features(df, circular_forecast_cols):
    """
    예보 데이터의 순환적 특성 처리 (풍향 등)
    
    Args:
        df: 데이터프레임
        circular_forecast_cols: 순환적 특성을 가진 예보 컬럼 목록 (wd_14, wd_20 등)
    
    Returns:
        처리된 데이터프레임
    """
    # 각 순환적 특성 처리
    for col in circular_forecast_cols:
        if col in df.columns:
            # 범위 확인 (풍향은 보통 0-360도)
            max_val = df[col].max()
            if pd.notna(max_val):  # NaN 체크
                if max_val <= 360:
                    # 사인/코사인 변환
                    df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / 360)
                    df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / 360)
                # 다른 범위인 경우 조정
                else:
                    df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / max_val)
                    df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    
    return df

'''def handle_missing_values(df, numeric_cols, zero_impute_cols, categorical_cols, circular_cols):
    """
    Handle missing values based on the provided strategy
    
    Strategy:
    1. For numeric weather features: Consider time pattern and use forward filling, 
       then anomaly imputation (median)
    2. For rn, icsr: Fill NaN with 0
    3. For categorical values: Forward fill, then mode imputation
    4. For circular features (wd_sin, wd_cos): Handle as numeric
    """
    
    # First identify all columns that exist in the dataframe
    existing_numeric_cols = [col for col in numeric_cols if col in df.columns]
    existing_zero_impute_cols = [col for col in zero_impute_cols if col in df.columns]
    existing_cat_cols = [col for col in categorical_cols if col in df.columns]
    existing_circular_cols = [col for col in circular_cols if col in df.columns]
    
    # Group data by hour to capture daily patterns (if datetime exists)
    if 'datetime' in df.columns and 'hour' in df.columns:
        # Handle numeric columns with time-based strategy
        for col in existing_numeric_cols + existing_circular_cols:
            if col not in df.columns:
                continue
                
            # First try forward fill (for small gaps)
            df[col] = df.groupby(['hour'])[col].transform(lambda x: x.ffill())
            
            # For remaining NaNs, use median by hour
            hourly_medians = df.groupby(['hour'])[col].transform('median')
            df[col] = df[col].fillna(hourly_medians)
            
            # If still NaN, use overall median
            df[col] = df[col].fillna(df[col].median())
    else:
        # Fallback to simple median imputation
        for col in existing_numeric_cols + existing_circular_cols:
            if col in df.columns:
                df[col] = df[col].fillna(df[col].median())
    
    # Handle zero-imputation columns
    for col in existing_zero_impute_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0)
    
    # Handle categorical columns
    for col in existing_cat_cols:
        if col not in df.columns:
            continue
            
        # First try forward fill 
        df[col] = df[col].ffill()
        
        # For remaining NaNs, use mode
        if not df[col].mode().empty:
            df[col] = df[col].fillna(df[col].mode().iloc[0])
        else:
            df[col] = df[col].fillna('unknown')
    
    return df
'''

def handle_missing_values(df, numeric_cols, zero_impute_cols, categorical_cols, circular_cols):
    """
    Handle missing values based on the provided strategy
    
    Strategy:
    1. For numeric weather features: 
       a. First try linear interpolation for whole dataset
       b. Then consider time pattern and use forward filling for remaining gaps
       c. Finally use anomaly imputation (median) if still missing
    2. For rn, icsr: Fill NaN with 0
    3. For categorical values: Forward fill, then mode imputation
    4. For circular features (wd_sin, wd_cos): Handle as numeric
    """
    
    # First identify all columns that exist in the dataframe
    existing_numeric_cols = [col for col in numeric_cols if col in df.columns]
    existing_zero_impute_cols = [col for col in zero_impute_cols if col in df.columns]
    existing_cat_cols = [col for col in categorical_cols if col in df.columns]
    existing_circular_cols = [col for col in circular_cols if col in df.columns]
    
    # Handle numeric and circular columns
    for col in existing_numeric_cols + existing_circular_cols:
        if col not in df.columns:
            continue

        # Step 0: 강제로 float 변환
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('float64')     
        
        # Step 1: Linear interpolation on the entire dataset
        df[col] = df[col].interpolate(method='linear', limit_direction='both')
        
        # If there are still NaNs and we have datetime information
        if df[col].isna().any() and 'datetime' in df.columns and 'hour' in df.columns:
            # Step 2: Forward fill 
            df = df.sort_values(by=['datetime'])
            df[col] = df[col].ffil()
            #df[col] = df.groupby('hour')[col].transform(lambda x: x.ffill())
            
            # Step 3: Use hour-specific medians for remaining NaNs
            hourly_medians = df.groupby('hour')[col].transform('median')
            df[col] = df[col].fillna(hourly_medians)
        
        # Step 4: If still NaN, use overall median
        overall_median = df[col].median()
        if pd.notna(overall_median):
            df[col] = df[col].fillna(overall_median)
    
    # Handle zero-imputation columns
    for col in existing_zero_impute_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0)
    
    # Handle categorical columns
    for col in existing_cat_cols:
        if col not in df.columns:
            continue
            
        # First try forward fill 
        df[col] = df[col].ffill()
        
        # For remaining NaNs, use mode
        if not df[col].mode().empty:
            df[col] = df[col].fillna(df[col].mode().iloc[0])
        else:
            df[col] = df[col].fillna('unknown')
    
    return df

def prepare_forecast_data(processed_data, feature_cols, forecast_cols_14, forecast_cols_20, 
                         target_col='value', history_days=1, include_forecast=True, meta_data=None):
    """
    예보 데이터와 메타데이터를 포함한 시계열 예측 데이터 준비
    
    Args:
        processed_data: 전처리된 데이터 딕셔너리
        feature_cols: 입력 피처 칼럼 목록 (예보 제외)
        forecast_cols_14: 14시 예보 컬럼 목록
        forecast_cols_20: 20시 예보 컬럼 목록
        target_col: 예측할 타겟 칼럼 ('value')
        history_days: 과거 데이터를 몇 일치 포함할지 (기본값: 1)
        include_forecast: 예보 데이터 포함 여부
        meta_data: 발전소 메타데이터 DataFrame
        
    Returns:
        forecast_data: 예보 데이터와 메타데이터를 포함한 시계열 데이터 딕셔너리
    """
    forecast_data = {
        '14시': {},  # 14시 예측 시나리오
        '20시': {}   # 20시 예측 시나리오
    }

    # 컬럼 정보 저장 딕셔너리
    column_info = {
        '14시': {
            'historical_cols': feature_cols + forecast_cols_14,                   # 과거 데이터 컬럼 (기본 특성, 'value' 포함)
            'base_feature_cols': feature_cols,                 # 기본 특성 컬럼만
            'forecast_cols': forecast_cols_14                  # 예보 컬럼만
        },
        '20시': {
            'historical_cols': feature_cols + forecast_cols_20,                   # 과거 데이터 컬럼 (기본 특성, 'value' 포함)
            'base_feature_cols': feature_cols,                 # 기본 특성 컬럼만
            'forecast_cols': forecast_cols_20                  # 예보 컬럼만
        }
    }
    
    # 메타데이터 처리 - 발전소별 정보 추출
    plant_meta = {}
    if meta_data is not None:
        for _, row in meta_data.iterrows():
            plant_name = row['name']
            plant_meta[plant_name] = {
                'capacity_kw': float(row['용량(MW)']) * 1000,  # MW -> kW 변환
                'year_built': int(row['준공년도']),
                'plant_age': 2025 - int(row['준공년도']),  # 현재 연도 기준 발전소 나이
                'region': row['광역시'] if pd.notna(row['광역시']) else "Unknown",
                'city': row['city'] if pd.notna(row['city']) else "Unknown",
                'district': row['district'] if pd.notna(row['district']) else "Unknown"
            }
    
    # 각 발전소별 데이터 처리
    for plant_id, df in processed_data.items():
        if 'datetime' not in df.columns or target_col not in df.columns:
            continue
        
        # 날짜별로 데이터 정렬
        df = df.sort_values('datetime')
        
        # 날짜만 추출하여 고유한 날짜 목록 생성
        if 'date_only' not in df.columns:
            df['date_only'] = df['datetime'].dt.date
            
        unique_dates = df['date_only'].unique()
        
        if len(unique_dates) <= history_days + 1:  # 최소 (history_days + 1)일 이상의 데이터가 필요
            print(f"Plant {plant_id} has insufficient data: {len(unique_dates)} days")
            continue
        
        # 각 날짜별로 24시간 데이터 그룹화
        date_groups = {}
        for date in unique_dates:
            date_df = df[df['date_only'] == date]
            if len(date_df) == 24:  # 하루에 24시간 데이터가 모두 있는 경우만 사용
                date_groups[date] = date_df.sort_values('hour')
        
        # 날짜 그룹을 시간순으로 정렬
        sorted_dates = sorted(date_groups.keys())
        
        # 메타데이터 추출
        meta_features = {}
        if plant_id in plant_meta:
            meta_features = plant_meta[plant_id]
        else:
            # 메타데이터가 없는 경우 기본값 설정
            meta_features = {
                'capacity_kw': 0.0,
                'year_built': 0,
                'plant_age': 0,
                'region': "Unknown",
                'city': "Unknown",
                'district': "Unknown"
            }
        
        # 두 예측 시나리오에 대한 데이터 준비
        for scenario in ['14시', '20시']:
            cutoff_hour = 14 if scenario == '14시' else 20
            
            # 시나리오에 맞는 예보 컬럼만 사용
            forecast_cols = forecast_cols_14 if scenario == '14시' else forecast_cols_20
            
            # 과거 데이터에 포함할 특성 (기본 특성 + value)
            historical_feature_cols = feature_cols + forecast_cols
            
            # 시나리오별 데이터 저장용 (딕셔너리 형태로 저장)
            samples = []
            
            # 각 예측 대상 날짜에 대해 데이터 생성
            for i in range(len(sorted_dates) - history_days - 1):
                # 과거 데이터 날짜들 (D-history_days ~ D-1)
                history_dates = sorted_dates[i:i+history_days]
                
                # 전날 (D-1)
                prev_date = sorted_dates[i+history_days-1]
                
                # 예측 대상일 (D-day)
                target_date = sorted_dates[i+history_days]
                
                # 과거 데이터 (D-history_days ~ D-2)
                history_data = []
                for hist_date in history_dates[:-1]:  # 전전날까지
                    if hist_date in date_groups:
                        # 과거 데이터에 예보 정보 포함 (+ value 포함)
                        history_data.append(date_groups[hist_date][historical_feature_cols].values)
                
                # 전날 데이터 (D-1)
                prev_data = date_groups[prev_date]
                
                # 전날 cutoff_hour 이전 데이터만 사용
                prev_data_cutoff = prev_data[prev_data['hour'] < cutoff_hour]
                
                # 전날 데이터에도 예보 정보 포함 (+ value 포함)
                prev_with_padding = prev_data_cutoff[historical_feature_cols].values
                
                # 패딩 추가 (전날 cutoff_hour ~ 23시까지)
                if cutoff_hour < 24:
                    # 패딩 크기 계산
                    padding_size = 24 - cutoff_hour
                    
                    # 0으로 채운 패딩 생성
                    padding = np.zeros((padding_size, len(historical_feature_cols)))
                    
                    # 전날 데이터와 패딩 결합
                    prev_with_padding = np.vstack([prev_with_padding, padding])
                
                # 과거 데이터 리스트 생성 (D-history_days ~ D-2, D-1+패딩)
                X_historical_parts = history_data + [prev_with_padding]
                
                # 과거 데이터 결합
                X_historical = np.vstack(X_historical_parts)  # 형태: (history_days*24, feature_dim)
                
                # 타겟 데이터 (D-day)
                target_data = date_groups[target_date]
                
                # 예보 데이터 추가 (시나리오에 맞는 예보 컬럼만 사용)
                # 여기서는 D-day에 대한 예보 데이터만 포함 (과거 예보는 이미 X_historical에 포함됨)
                if include_forecast and forecast_cols:
                    # 예측 대상일(D-day)의 예보 데이터
                    X_forecast = target_data[forecast_cols].values  # 형태: (24, forecast_dim)
                else:
                    # 예보 데이터가 없는 경우 빈 배열
                    X_forecast = np.zeros((24, len(forecast_cols) if forecast_cols else 0))
                
                # 메타데이터 처리
                # 메타데이터는 각 샘플에 대해 동일한 값으로 사용됨
                X_meta = np.array([
                    float(meta_features['capacity_kw']),
                    float(meta_features['plant_age']),
                    meta_features['region'],
                ])
                
                # 지역 정보를 원-핫 인코딩으로 처리하려면 별도 처리 필요
                # 현재는 간단하게 숫자 특성만 사용
                
                # 데이터 딕셔너리로 저장
                sample = {
                    'X_historical': X_historical,
                    'X_forecast': X_forecast,
                    'X_meta': X_meta,
                    'y': target_data[target_col].values,
                    'dates': target_data['datetime'].values
                }
                
                samples.append(sample)
            
            # 발전소별, 시나리오별 데이터 저장
            if samples:
                forecast_data[scenario][plant_id] = samples
    
    # 메타데이터 컬럼 이름 저장
    meta_column_info = {
        'meta_cols': ['capacity_kw', 'plant_age', 'region']
    }
    
    # 컬럼 정보에 메타데이터 컬럼 추가
    for scenario in column_info:
        column_info[scenario].update(meta_column_info)
    
    return forecast_data, column_info

def split_forecast_data(forecast_data, plants_info, external_test_ratio=0.2, train_ratio=0.7, valid_ratio=0.15):
    """
    예보 데이터를 포함한 시계열 데이터를 학습/검증/테스트 세트로 분할
    
    전략:
    1. 일부 발전소(데이터가 적은)를 external_test_plants로 분리 - 모델 학습에 전혀 사용하지 않음
    2. 나머지 발전소 데이터는 시간 순서대로 train/valid/test로 분할
    
    Args:
        forecast_data: 예보 데이터를 포함한 시계열 데이터 딕셔너리 {'14시': {...}, '20시': {...}}
        plants_info: 각 발전소 정보를 담은 딕셔너리
        external_test_ratio: 외부 테스트용 발전소 비율
        train_ratio: 학습 데이터 비율 (external test를 제외한 나머지에서)
        valid_ratio: 검증 데이터 비율 (external test를 제외한 나머지에서)
        
    Returns:
        split_forecast_data: 분할된 데이터 딕셔너리
    """
    # 시나리오별 분할 데이터 저장 딕셔너리
    split_forecast_data = {}
    
    # 데이터가 있는 발전소 식별
    available_plants = set()
    for scenario in forecast_data:
        for plant_id in forecast_data[scenario]:
            available_plants.add(plant_id)
    
    # 발전소를 데이터 길이에 따라 정렬 (오름차순 - 데이터가 적은 순)
    sorted_plants = sorted(
        list(available_plants), 
        key=lambda p: plants_info[p]['data_length'] if 'data_length' in plants_info[p] else 0
    )
    
    # 외부 테스트용 발전소 선택 (데이터가 적은 발전소들)
    n_external_test = max(1, int(len(sorted_plants) * external_test_ratio))
    external_test_plants = sorted_plants[:n_external_test]
    train_valid_test_plants = sorted_plants[n_external_test:]
    
    print(f"External test plants: {external_test_plants}")
    print(f"Regular split plants: {train_valid_test_plants}")
    
    # 각 시나리오별 데이터 분할
    for scenario in forecast_data:
        split_data = {
            'train': {
                'X_historical': [], 'X_forecast': [], 'X_meta': [], 'y': [], 'dates': [], 'plants': []
            },
            'valid': {
                'X_historical': [], 'X_forecast': [], 'X_meta': [], 'y': [], 'dates': [], 'plants': []
            },
            'test': {
                'X_historical': [], 'X_forecast': [], 'X_meta': [], 'y': [], 'dates': [], 'plants': []
            },
            'external_test': {
                'X_historical': [], 'X_forecast': [], 'X_meta': [], 'y': [], 'dates': [], 'plants': []
            }
        }
        
        # 외부 테스트 발전소 데이터 분리
        for plant_id in external_test_plants:
            if plant_id not in forecast_data[scenario]:
                continue
                
            samples = forecast_data[scenario][plant_id]
            n_samples = len(samples)
            
            for sample in samples:
                split_data['external_test']['X_historical'].append(sample['X_historical'])
                split_data['external_test']['X_forecast'].append(sample['X_forecast'])
                split_data['external_test']['X_meta'].append(sample['X_meta'])
                split_data['external_test']['y'].append(sample['y'])
                split_data['external_test']['dates'].append(sample['dates'])
                split_data['external_test']['plants'].append(plant_id)
        
        # 나머지 발전소 데이터를 시간 순서대로 분할
        for plant_id in train_valid_test_plants:
            if plant_id not in forecast_data[scenario]:
                continue
                
            samples = forecast_data[scenario][plant_id]
            n_samples = len(samples)
            if n_samples < 3:  # 최소 3개 이상의 샘플이 필요
                continue
                
            # 인덱스 계산
            train_idx = int(n_samples * train_ratio)
            valid_idx = int(n_samples * (train_ratio + valid_ratio))
            
            # 데이터 분할 - 시간 순서대로 분할
            for i, sample in enumerate(samples):
                if i < train_idx:
                    # Train set
                    split_data['train']['X_historical'].append(sample['X_historical'])
                    split_data['train']['X_forecast'].append(sample['X_forecast'])
                    split_data['train']['X_meta'].append(sample['X_meta'])
                    split_data['train']['y'].append(sample['y'])
                    split_data['train']['dates'].append(sample['dates'])
                    split_data['train']['plants'].append(plant_id)
                elif i < valid_idx:
                    # Validation set
                    split_data['valid']['X_historical'].append(sample['X_historical'])
                    split_data['valid']['X_forecast'].append(sample['X_forecast'])
                    split_data['valid']['X_meta'].append(sample['X_meta'])
                    split_data['valid']['y'].append(sample['y'])
                    split_data['valid']['dates'].append(sample['dates'])
                    split_data['valid']['plants'].append(plant_id)
                else:
                    # Test set
                    split_data['test']['X_historical'].append(sample['X_historical'])
                    split_data['test']['X_forecast'].append(sample['X_forecast'])
                    split_data['test']['X_meta'].append(sample['X_meta'])
                    split_data['test']['y'].append(sample['y'])
                    split_data['test']['dates'].append(sample['dates'])
                    split_data['test']['plants'].append(plant_id)
        
        # 배열로 변환
        for split_name in split_data:
            if split_data[split_name]['X_historical']:  # 비어있지 않은 경우만 처리
                split_data[split_name]['X_historical'] = np.array(split_data[split_name]['X_historical'])
                split_data[split_name]['X_forecast'] = np.array(split_data[split_name]['X_forecast'])
                split_data[split_name]['X_meta'] = np.array(split_data[split_name]['X_meta'])
                split_data[split_name]['y'] = np.array(split_data[split_name]['y'])
                split_data[split_name]['dates'] = np.array(split_data[split_name]['dates'])
                # plants는 리스트로 유지 (식별자이므로 변환 불필요)
        
        # 시나리오별 분할 데이터 저장
        split_forecast_data[scenario] = split_data
    
    return split_forecast_data

def save_forecast_data(split_forecast_data, output_dir='../data/forecast_modeling_data'):
    """
    분할된 예보 데이터 저장
    
    Args:
        split_forecast_data: 분할된 예보 데이터 딕셔너리 (시나리오별)
        output_dir: 저장할 디렉토리 경로
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for scenario, split_data in split_forecast_data.items():
        scenario_dir = os.path.join(output_dir, scenario)
        os.makedirs(scenario_dir, exist_ok=True)
        
        for split_name, data in split_data.items():
            if data['X_historical'].size == 0:  # 빈 데이터 세트 건너뛰기
                continue
                
            split_dir = os.path.join(scenario_dir, split_name)
            os.makedirs(split_dir, exist_ok=True)
            
            # 데이터 저장
            np.save(os.path.join(split_dir, 'X_historical.npy'), data['X_historical'])
            np.save(os.path.join(split_dir, 'X_forecast.npy'), data['X_forecast'])
            np.save(os.path.join(split_dir, 'X_meta.npy'), data['X_meta'])
            np.save(os.path.join(split_dir, 'y.npy'), data['y'])
            np.save(os.path.join(split_dir, 'dates.npy'), data['dates'])
            np.save(os.path.join(split_dir, 'plants.npy'), np.array(data['plants']))
            
            print(f"Saved {split_name} data for {scenario} scenario:")
            print(f"  X_historical shape: {data['X_historical'].shape}")
            print(f"  X_forecast shape: {data['X_forecast'].shape}")
            print(f"  X_meta shape: {data['X_meta'].shape}")
            print(f"  y shape: {data['y'].shape}")


# Define paths
META_DATA_PATH = '../data/solar_energy/meta_data.csv'
data_dir = '../data/concat_forecast_data'
output_dir = '../data/forecast_modeling_data'

# 과거 데이터 일수, 외부 테스트용 발전소 비율
history_days = 2  # 전날 + 전전날 데이터 사용
external_test_ratio = 0.2

# 데이터 로드 및 전처리
processed_data, feature_cols, forecast_cols_14, forecast_cols_20, target_col, plants_info, meta_data = load_and_preprocess_data(
    META_DATA_PATH, data_dir
)

# 결과 요약 출력
print(f"Processed {len(processed_data)} plants")
print(f"Base feature columns (excluding forecast): {len(feature_cols)}")
print(f"14시 예보 컬럼: {forecast_cols_14}")
print(f"20시 예보 컬럼: {forecast_cols_20}")
print(f"Target column: {target_col}")


Available 14시 forecast columns: ['temp_14', 'wd_14', 'sc_14', 'ws_14', 'pp_14']
Available 20시 forecast columns: ['temp_20', 'wd_20', 'sc_20', 'ws_20', 'pp_20']


100%|██████████| 37/37 [00:00<00:00, 124.67it/s]


Found 37 plants
Processing each plant's data...


  3%|▎         | 1/37 [00:00<00:05,  7.05it/s]

===== Missing Values Before Processing =====
rn: 27716개 (90.3%)
ws: 30개 (0.1%)
wd: 30개 (0.1%)
ss: 13865개 (45.2%)
icsr: 15108개 (49.2%)
dc10Tca: 1163개 (3.8%)
dc10LmcsCa: 81개 (0.3%)
lcsCh: 16357개 (53.3%)
vs: 8개 (0.0%)
SO2: 1171개 (3.8%)
CO: 466개 (1.5%)
O3: 425개 (1.4%)
NO2: 438개 (1.4%)
PM10: 345개 (1.1%)
PM25: 378개 (1.2%)
초미세먼지: 378개 (1.2%)
temp_14: 24개 (0.1%)
wd_14: 24개 (0.1%)
ws_14: 24개 (0.1%)
temp_20: 24개 (0.1%)
wd_20: 24개 (0.1%)
ws_20: 24개 (0.1%)
wd_sin: 30개 (0.1%)
wd_cos: 30개 (0.1%)
wd_14_sin: 24개 (0.1%)
wd_14_cos: 24개 (0.1%)
wd_20_sin: 24개 (0.1%)
wd_20_cos: 24개 (0.1%)

===== Missing Values After Processing =====
모든 결측치가 성공적으로 처리되었습니다. NaN 값이 없습니다.
===== Missing Values Before Processing =====
temperature: 53개 (0.4%)
humidity: 74개 (0.6%)
rn: 11574개 (93.1%)
ws: 54개 (0.4%)
wd: 54개 (0.4%)
pv: 74개 (0.6%)
pa: 53개 (0.4%)
ps: 53개 (0.4%)
ss: 5750개 (46.3%)
icsr: 5750개 (46.3%)
dc10Tca: 112개 (0.9%)
dc10LmcsCa: 165개 (1.3%)
lcsCh: 6115개 (49.2%)
vs: 152개 (1.2%)
ts: 55개 (0.4%)
SO2: 896개 (7.2%)
CO: 893개

...


===== Missing Values After Processing =====
모든 결측치가 성공적으로 처리되었습니다. NaN 값이 없습니다.
===== Missing Values Before Processing =====
temperature: 53개 (0.4%)
humidity: 74개 (0.6%)
rn: 11574개 (93.1%)
ws: 54개 (0.4%)
wd: 54개 (0.4%)
pv: 74개 (0.6%)
pa: 53개 (0.4%)
ps: 53개 (0.4%)
ss: 5750개 (46.3%)
icsr: 5750개 (46.3%)
dc10Tca: 112개 (0.9%)
dc10LmcsCa: 165개 (1.3%)
lcsCh: 6115개 (49.2%)
vs: 152개 (1.2%)
ts: 55개 (0.4%)
SO2: 896개 (7.2%)
CO: 893개 (7.2%)
O3: 894개 (7.2%)
NO2: 893개 (7.2%)
PM10: 943개 (7.6%)
PM25: 945개 (7.6%)
초미세먼지: 945개 (7.6%)
temp_20: 1036개 (8.3%)
wd_20: 1036개 (8.3%)
ws_20: 1036개 (8.3%)
pp_20: 1036개 (8.3%)
wd_sin: 54개 (0.4%)
wd_cos: 54개 (0.4%)
wd_20_sin: 1036개 (8.3%)
wd_20_cos: 1036개 (8.3%)

===== Missing Values After Processing =====
모든 결측치가 성공적으로 처리되었습니다. NaN 값이 없습니다.
===== Missing Values Before Processing =====
rn: 27716개 (90.3%)
ws: 30개 (0.1%)
wd: 30개 (0.1%)
ss: 13865개 (45.2%)
icsr: 15108개 (49.2%)
dc10Tca: 1163개 (3.8%)
dc10LmcsCa: 81개 (0.3%)
lcsCh: 16357개 (53.3%)
vs: 8개 (0.0%)
SO2: 683개 (2.2%

 95%|█████████▍| 35/37 [00:03<00:00, 11.87it/s]

ss: 3705개 (45.9%)
icsr: 3705개 (45.9%)
dc10Tca: 2개 (0.0%)
dc10LmcsCa: 2개 (0.0%)
lcsCh: 4176개 (51.8%)
SO2: 217개 (2.7%)
CO: 211개 (2.6%)
O3: 217개 (2.7%)
NO2: 277개 (3.4%)
PM10: 199개 (2.5%)
PM25: 362개 (4.5%)
초미세먼지: 362개 (4.5%)
temp_14: 24개 (0.3%)
wd_14: 24개 (0.3%)
ws_14: 24개 (0.3%)
temp_20: 24개 (0.3%)
wd_20: 24개 (0.3%)
ws_20: 24개 (0.3%)
wd_14_sin: 24개 (0.3%)
wd_14_cos: 24개 (0.3%)
wd_20_sin: 24개 (0.3%)
wd_20_cos: 24개 (0.3%)

===== Missing Values After Processing =====
모든 결측치가 성공적으로 처리되었습니다. NaN 값이 없습니다.
===== Missing Values Before Processing =====
rn: 7184개 (89.1%)
ss: 3705개 (45.9%)
icsr: 3705개 (45.9%)
dc10Tca: 2개 (0.0%)
dc10LmcsCa: 2개 (0.0%)
lcsCh: 4176개 (51.8%)
SO2: 217개 (2.7%)
CO: 211개 (2.6%)
O3: 217개 (2.7%)
NO2: 277개 (3.4%)
PM10: 199개 (2.5%)
PM25: 362개 (4.5%)
초미세먼지: 362개 (4.5%)
temp_14: 24개 (0.3%)
wd_14: 24개 (0.3%)
ws_14: 24개 (0.3%)
temp_20: 24개 (0.3%)
wd_20: 24개 (0.3%)
ws_20: 24개 (0.3%)
wd_14_sin: 24개 (0.3%)
wd_14_cos: 24개 (0.3%)
wd_20_sin: 24개 (0.3%)
wd_20_cos: 24개 (0.3%)

===== Missing

100%|██████████| 37/37 [00:03<00:00, 11.11it/s]


===== Missing Values After Processing =====
모든 결측치가 성공적으로 처리되었습니다. NaN 값이 없습니다.
===== Missing Values Before Processing =====
rn: 19854개 (90.4%)
ws: 30개 (0.1%)
wd: 30개 (0.1%)
ss: 9911개 (45.1%)
icsr: 9919개 (45.2%)
dc10Tca: 37개 (0.2%)
dc10LmcsCa: 30개 (0.1%)
lcsCh: 11707개 (53.3%)
vs: 7개 (0.0%)
SO2: 551개 (2.5%)
CO: 537개 (2.4%)
O3: 534개 (2.4%)
NO2: 598개 (2.7%)
PM10: 658개 (3.0%)
PM25: 763개 (3.5%)
초미세먼지: 763개 (3.5%)
temp_14: 24개 (0.1%)
wd_14: 24개 (0.1%)
ws_14: 24개 (0.1%)
temp_20: 24개 (0.1%)
wd_20: 24개 (0.1%)
ws_20: 24개 (0.1%)
wd_sin: 30개 (0.1%)
wd_cos: 30개 (0.1%)
wd_14_sin: 24개 (0.1%)
wd_14_cos: 24개 (0.1%)
wd_20_sin: 24개 (0.1%)
wd_20_cos: 24개 (0.1%)

===== Missing Values After Processing =====
모든 결측치가 성공적으로 처리되었습니다. NaN 값이 없습니다.
Base feature columns (excluding forecast): 41
14시 forecast columns: 5
20시 forecast columns: 5
Processed 37 plants
Base feature columns (excluding forecast): 41
14시 예보 컬럼: ['temp_14', 'wd_14', 'sc_14', 'ws_14', 'pp_14']
20시 예보 컬럼: ['temp_20', 'wd_20', 'sc_20', 'ws_20',

In [30]:
# 메타데이터 요약 출력
print(f"Loaded metadata for {len(meta_data)} plants")
print(f"Metadata columns: {meta_data.columns.tolist()}")

# 예보 데이터와 메타데이터를 포함한 시계열 데이터 준비
forecast_data, column_info = prepare_forecast_data(
    processed_data, 
    feature_cols, 
    forecast_cols_14, 
    forecast_cols_20,
    target_col=target_col,
    history_days=history_days,
    include_forecast=True,
    meta_data=meta_data
)

os.makedirs(output_dir, exist_ok=True)

# 각 시나리오별 컬럼 정보를 JSON 파일로 저장
for scenario, cols in column_info.items():
    # 리스트를 JSON으로 직렬화 가능하게 변환
    serializable_cols = {
        'historical_cols': list(cols['historical_cols']),
        'base_feature_cols': list(cols['base_feature_cols']),
        'forecast_cols': list(cols['forecast_cols']),
        'meta_cols': list(cols['meta_cols'])  # 메타데이터 컬럼 추가
    }
    
    # JSON 파일로 저장
    col_file = os.path.join(output_dir, f"{scenario}_columns.json")
    with open(col_file, 'w') as f:
        json.dump(serializable_cols, f, indent=2)
    
    print(f"Column information for {scenario} saved to {col_file}")

# 각 시나리오별 발전소 수 및 샘플 수 출력
for scenario, scenario_data in forecast_data.items():
    plant_count = len(scenario_data)
    total_samples = sum(len(samples) for plant_id, samples in scenario_data.items())
    
    print(f"\n{scenario} 시나리오:")
    print(f"  발전소 수: {plant_count}")
    print(f"  총 샘플 수: {total_samples}")
    
    # 각 발전소별 샘플 수 출력 (처음 5개만)
    for i, (plant_id, samples) in enumerate(scenario_data.items()):
        if i >= 5:  # 처음 5개 발전소만 출력
            break
            
        print(f"  발전소 {plant_id}: {len(samples)} 샘플")
        if len(samples) > 0:
            print(f"    X_historical 형태: {samples[0]['X_historical'].shape}")
            print(f"    X_forecast 형태: {samples[0]['X_forecast'].shape}")
            print(f"    X_meta 형태: {samples[0]['X_meta'].shape}")
            print(f"    y 형태: {samples[0]['y'].shape}")

# 데이터 분할 (학습/검증/테스트 + 외부 테스트)
split_forecast_data = split_forecast_data(
    forecast_data, 
    plants_info,
    external_test_ratio=external_test_ratio
)

# 각 시나리오별, 세트별 결과 요약 출력
for scenario, split_data in split_forecast_data.items():
    print(f"\n{scenario.upper()} 시나리오:")
    
    for split_name, data in split_data.items():
        if data['X_historical'].size == 0:
            continue
            
        print(f"  {split_name.upper()} 세트:")
        print(f"    샘플 수: {len(data['X_historical'])}")
        print(f"    X_historical 형태: {data['X_historical'].shape}")
        print(f"    X_forecast 형태: {data['X_forecast'].shape}")
        print(f"    X_meta 형태: {data['X_meta'].shape}")
        print(f"    y 형태: {data['y'].shape}")
        
        # 발전소 정보
        plant_counts = {}
        for plant_id in data['plants']:
            if plant_id in plant_counts:
                plant_counts[plant_id] += 1
            else:
                plant_counts[plant_id] = 1
        
        print(f"    발전소 수: {len(plant_counts)}")
        print(f"    발전소 분포: {plant_counts}")

# 데이터 저장
save_forecast_data(split_forecast_data, output_dir)
print(f"\n예보 데이터와 메타데이터가 포함된 모델링 데이터 저장 완료: {output_dir}")

Loaded metadata for 46 plants
Metadata columns: ['name', '발전소명', '구분', '용량(MW)', '준공년도', '비 고', 'location_name', '광역시', 'city', 'district']
Column information for 14시 saved to ../data/forecast_modeling_data/14시_columns.json
Column information for 20시 saved to ../data/forecast_modeling_data/20시_columns.json

14시 시나리오:
  발전소 수: 37
  총 샘플 수: 24741
  발전소 부산운동장: 1276 샘플
    X_historical 형태: (48, 46)
    X_forecast 형태: (24, 5)
    X_meta 형태: (3,)
    y 형태: (24,)
  발전소 화촌주민참여형: 515 샘플
    X_historical 형태: (48, 46)
    X_forecast 형태: (24, 5)
    X_meta 형태: (3,)
    y 형태: (24,)
  발전소 삼척소내_2: 879 샘플
    X_historical 형태: (48, 46)
    X_forecast 형태: (24, 5)
    X_meta 형태: (3,)
    y 형태: (24,)
  발전소 영월본부: 911 샘플
    X_historical 형태: (48, 46)
    X_forecast 형태: (24, 5)
    X_meta 형태: (3,)
    y 형태: (24,)
  발전소 하동변전소: 515 샘플
    X_historical 형태: (48, 46)
    X_forecast 형태: (24, 5)
    X_meta 형태: (3,)
    y 형태: (24,)

20시 시나리오:
  발전소 수: 37
  총 샘플 수: 24741
  발전소 부산운동장: 1276 샘플
    X_historical 형태: (48,

In [31]:
def load_column_info(scenario, column_file):
    with open(column_file, 'r') as f:
        return json.load(f)

# 14시 시나리오의 컬럼 정보 로드
scenario = '14시'
column_info = load_column_info(scenario, f"../data/forecast_modeling_data/{scenario}_columns.json")

# 컬럼 정보 활용
historical_cols = column_info['historical_cols']
base_feature_cols = column_info['base_feature_cols']
forecast_cols = column_info['forecast_cols']
meta_cols = column_info.get('meta_cols', [])  # meta_cols가 없을 경우 빈 리스트 반환

print(f"Historical columns ({len(historical_cols)}): {historical_cols}")
print(f"Forecast columns ({len(forecast_cols)}): {forecast_cols}")
print(f"Meta columns ({len(meta_cols)}): {meta_cols}")

Historical columns (46): ['총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별_kw)', '최소(시간별_kw)', 'value', 'temperature', 'humidity', 'rn', 'ws', 'wd', 'pv', 'pa', 'ps', 'ss', 'icsr', 'dc10Tca', 'dc10LmcsCa', 'lcsCh', 'vs', 'ts', 'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25', '미세먼지', '초미세먼지', 'time_hour_sin', 'time_hour_cos', 'time_day_sin', 'time_day_cos', 'time_month_sin', 'time_month_cos', 'time_is_daylight', 'time_hours_since_sunrise', 'time_hours_until_sunset', 'wd_sin', 'wd_cos', 'temp_14', 'wd_14', 'sc_14', 'ws_14', 'pp_14']
Forecast columns (5): ['temp_14', 'wd_14', 'sc_14', 'ws_14', 'pp_14']
Meta columns (3): ['capacity_kw', 'plant_age', 'region']


In [32]:
'총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별_kw)', '최소(시간별_kw)', 'value',
'temperature', 'humidity', 'rn', 'ws', 'wd', 'pv', 'pa', 'ps', 'ss', 'icsr', 
'dc10Tca', 'dc10LmcsCa', 'lcsCh', 'vs', 'ts', 
'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25', '미세먼지', '초미세먼지', 
'time_hour_sin', 'time_hour_cos', 'time_day_sin', 'time_day_cos', 'time_month_sin', 'time_month_cos', 
'time_is_daylight', 'time_hours_since_sunrise', 'time_hours_until_sunset', 'wd_sin', 'wd_cos', 
'temp_14', 'wd_14', 'sc_14', 'ws_14', 'pp_14'

('temp_14', 'wd_14', 'sc_14', 'ws_14', 'pp_14')

In [33]:
a = np.load('../data/forecast_modeling_data/20시/test/X_forecast.npy',allow_pickle=True)
print(a.shape)
a

(3423, 24, 5)


array([[[20.0, 94.0, '1.0', 1.0, 0.0],
        [21.0, 139.0, '1.0', 2.0, 0.0],
        [22.0, 185.0, '1.0', 2.0, 0.0],
        ...,
        [16.0, 117.0, '1.0', 1.0, 0.0],
        [16.0, 117.0, '1.0', 1.0, 0.0],
        [16.0, 117.0, '1.0', 1.0, 0.0]],

       [[20.0, 124.0, '1.0', 1.0, 0.0],
        [21.0, 146.0, '1.0', 2.0, 0.0],
        [23.0, 169.0, '1.0', 2.0, 0.0],
        ...,
        [19.0, 114.0, '4.0', 3.0, 30.0],
        [19.0, 114.0, '4.0', 3.0, 30.0],
        [19.0, 114.0, '4.0', 3.0, 30.0]],

       [[19.0, 145.0, '4.0', 3.0, 60.0],
        [19.0, 128.0, '4.0', 3.0, 60.0],
        [20.0, 112.0, '4.0', 2.0, 60.0],
        ...,
        [18.0, 45.0, '4.0', 2.0, 70.0],
        [18.0, 45.0, '4.0', 2.0, 70.0],
        [18.0, 45.0, '4.0', 2.0, 70.0]],

       ...,

       [[24.0, 52.0, '1.0', 3.0, 10.0],
        [25.0, 63.0, '1.0', 4.0, 10.0],
        [25.0, 73.0, '1.0', 4.0, 10.0],
        ...,
        [21.0, 41.0, '3.0', 3.0, 20.0],
        [21.0, 41.0, '3.0', 3.0, 20.0],
    

In [34]:
a = np.load('../data/forecast_modeling_data/20시/test/X_historical.npy',allow_pickle=True)
print(a.shape)
a

(3423, 48, 46)


array([[[3477.426, 144.893, 463.785, ..., '1.0', 3.0, 0.0],
        [3477.426, 144.893, 463.785, ..., '1.0', 3.0, 0.0],
        [3477.426, 144.893, 463.785, ..., '1.0', 3.0, 0.0],
        ...,
        [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]],

       [[3312.063, 138.003, 458.964, ..., '4.0', 3.0, 30.0],
        [3312.063, 138.003, 458.964, ..., '4.0', 3.0, 30.0],
        [3312.063, 138.003, 458.964, ..., '4.0', 2.0, 30.0],
        ...,
        [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]],

       [[3377.147, 140.714, 431.484, ..., '1.0', 1.0, 0.0],
        [3377.147, 140.714, 431.484, ..., '1.0', 2.0, 0.0],
        [3377.147, 140.714, 431.484, ..., '1.0', 2.0, 0.0],
        ...,
        [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]],

       

In [35]:
a = np.load('../data/forecast_modeling_data/20시/test/X_meta.npy',allow_pickle=True)
print(a.shape)
a

(3423, 3)


array([['603.0', '6.0', '인천광역시'],
       ['603.0', '6.0', '인천광역시'],
       ['603.0', '6.0', '인천광역시'],
       ...,
       ['390.0', '17.0', '부산'],
       ['390.0', '17.0', '부산'],
       ['390.0', '17.0', '부산']], dtype='<U32')

In [56]:
import numpy as np
import pandas as pd
from datetime import datetime

def check_date_ranges(base_path='../data/forecast_modeling_data'):
    """
    각 시나리오별, 데이터 세트별 날짜 범위 확인
    
    Args:
        base_path: 데이터 폴더 경로
    """
    
    scenarios = ['14시', '20시']
    splits = ['train', 'valid', 'test', 'external_test']
    
    print("=" * 80)
    print("데이터 세트별 날짜 범위 현황")
    print("=" * 80)
    
    for scenario in scenarios:
        print(f"\n🕐 {scenario} 시나리오")
        print("-" * 60)
        
        scenario_summary = []
        
        for split in splits:
            try:
                # dates.npy 파일 로드
                dates_path = f"{base_path}/{scenario}/{split}/dates.npy"
                dates = np.load(dates_path, allow_pickle=True)
                
                # numpy datetime64를 pandas datetime으로 변환
                if len(dates) > 0:
                    # 3D 배열인 경우 flatten
                    if dates.ndim > 1:
                        dates_flat = dates.flatten()
                    else:
                        dates_flat = dates
                    
                    # datetime 변환
                    dates_pd = pd.to_datetime(dates_flat)
                    
                    # 날짜만 추출 (시간 제거)
                    dates_only = dates_pd.date
                    unique_dates = sorted(set(dates_only))
                    
                    start_date = unique_dates[0]
                    end_date = unique_dates[-1]
                    total_days = len(unique_dates)
                    total_samples = len(dates_flat)
                    
                    print(f"📁 {split.upper():<12}")
                    print(f"   📅 날짜 범위: {start_date} ~ {end_date}")
                    print(f"   📊 총 일수: {total_days}일")
                    print(f"   📈 총 샘플: {total_samples:,}개")
                    print(f"   🔢 일평균 샘플: {total_samples/total_days:.1f}개/일")
                    print()
                    
                    scenario_summary.append({
                        'split': split,
                        'start_date': start_date,
                        'end_date': end_date,
                        'days': total_days,
                        'samples': total_samples
                    })
                    
            except FileNotFoundError:
                print(f"❌ {split.upper()}: 파일을 찾을 수 없습니다.")
            except Exception as e:
                print(f"❌ {split.upper()}: 오류 발생 - {e}")
        
        # 시나리오 요약
        if scenario_summary:
            print("📋 시나리오 요약:")
            total_days_scenario = sum(s['days'] for s in scenario_summary)
            total_samples_scenario = sum(s['samples'] for s in scenario_summary)
            
            print(f"   전체 기간: {min(s['start_date'] for s in scenario_summary)} ~ "
                  f"{max(s['end_date'] for s in scenario_summary)}")
            print(f"   총 데이터 일수: {total_days_scenario}일")
            print(f"   총 샘플 수: {total_samples_scenario:,}개")
            
            # 시간적 연속성 확인
            print("\n⏰ 시간적 분할 순서:")
            sorted_summary = sorted(scenario_summary, key=lambda x: x['start_date'])
            for i, s in enumerate(sorted_summary):
                print(f"   {i+1}. {s['split']}: {s['start_date']} ~ {s['end_date']}")

def check_temporal_consistency(base_path='../data/forecast_modeling_data', scenario='14시'):
    """
    시간적 일관성 상세 확인 (겹치는 날짜가 있는지 등)
    
    Args:
        base_path: 데이터 폴더 경로
        scenario: 확인할 시나리오
    """
    
    print(f"\n🔍 {scenario} 시나리오 시간적 일관성 상세 분석")
    print("=" * 60)
    
    splits = ['train', 'valid', 'test', 'external_test']
    split_dates = {}
    
    # 각 split별 날짜 수집
    for split in splits:
        try:
            dates_path = f"{base_path}/{scenario}/{split}/dates.npy"
            dates = np.load(dates_path, allow_pickle=True)
            
            if dates.ndim > 1:
                dates_flat = dates.flatten()
            else:
                dates_flat = dates
            
            dates_pd = pd.to_datetime(dates_flat)
            unique_dates = set(dates_pd.date)
            split_dates[split] = unique_dates
            
        except Exception as e:
            print(f"❌ {split}: {e}")
            split_dates[split] = set()
    
    # 겹치는 날짜 확인
    print("📅 Split 간 날짜 겹침 확인:")
    splits_with_data = [s for s in splits if split_dates[s]]
    
    overlap_found = False
    for i, split1 in enumerate(splits_with_data):
        for split2 in splits_with_data[i+1:]:
            overlap = split_dates[split1] & split_dates[split2]
            if overlap:
                overlap_found = True
                print(f"   ⚠️  {split1} ↔ {split2}: {len(overlap)}일 겹침")
                if len(overlap) <= 5:  # 5일 이하면 구체적으로 출력
                    print(f"      겹치는 날짜: {sorted(overlap)}")
    
    if not overlap_found:
        print("   ✅ 모든 split 간 날짜 겹침 없음 (정상)")
    
    # 날짜 간격 확인
    print(f"\n📈 Split 간 시간적 연속성:")
    sorted_splits = sorted([(split, min(dates), max(dates)) 
                           for split, dates in split_dates.items() if dates],
                          key=lambda x: x[1])
    
    for i, (split, start, end) in enumerate(sorted_splits):
        print(f"   {i+1}. {split}: {start} ~ {end}")
        if i > 0:
            prev_end = sorted_splits[i-1][2]
            gap = (start - prev_end).days - 1
            if gap > 0:
                print(f"      ⚠️  이전 split과 {gap}일 간격")
            elif gap < 0:
                print(f"      ⚠️  이전 split과 {abs(gap)}일 겹침")
            else:
                print(f"      ✅ 이전 split과 연속됨")

# 사용 예시
if __name__ == "__main__":
    # 전체 날짜 범위 확인
    check_date_ranges()
    
    # 상세 일관성 분석 (14시 시나리오)
    check_temporal_consistency(scenario='14시')
    
    print("\n" + "="*80)


데이터 세트별 날짜 범위 현황

🕐 14시 시나리오
------------------------------------------------------------
📁 TRAIN       
   📅 날짜 범위: 2015-01-04 ~ 2022-08-04
   📊 총 일수: 2759일
   📈 총 샘플: 381,480개
   🔢 일평균 샘플: 138.3개/일

📁 VALID       
   📅 날짜 범위: 2019-07-24 ~ 2022-10-17
   📊 총 일수: 812일
   📈 총 샘플: 81,744개
   🔢 일평균 샘플: 100.7개/일

📁 TEST        
   📅 날짜 범위: 2020-07-12 ~ 2022-12-30
   📊 총 일수: 487일
   📈 총 샘플: 82,152개
   🔢 일평균 샘플: 168.7개/일

📁 EXTERNAL_TEST
   📅 날짜 범위: 2020-08-03 ~ 2022-12-30
   📊 총 일수: 664일
   📈 총 샘플: 48,408개
   🔢 일평균 샘플: 72.9개/일

📋 시나리오 요약:
   전체 기간: 2015-01-04 ~ 2022-12-30
   총 데이터 일수: 4722일
   총 샘플 수: 593,784개

⏰ 시간적 분할 순서:
   1. train: 2015-01-04 ~ 2022-08-04
   2. valid: 2019-07-24 ~ 2022-10-17
   3. test: 2020-07-12 ~ 2022-12-30
   4. external_test: 2020-08-03 ~ 2022-12-30

🕐 20시 시나리오
------------------------------------------------------------
📁 TRAIN       
   📅 날짜 범위: 2015-01-04 ~ 2022-08-04
   📊 총 일수: 2759일
   📈 총 샘플: 381,480개
   🔢 일평균 샘플: 138.3개/일

📁 VALID       
   📅 날짜 범위: 2019-07-

### normalization

In [45]:
import os
import numpy as np
import pandas as pd
import json
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tqdm import tqdm

In [47]:

def load_column_info(input_dir, scenario):
    """컬럼 정보 로드"""
    col_file = os.path.join(input_dir, f"{scenario}_columns.json")
    with open(col_file, 'r') as f:
        return json.load(f)

def identify_column_types(columns):
    """
    특성 컬럼 유형 식별
    
    Args:
        columns: 컬럼 목록
    
    Returns:
        numerical_cols: 수치형 컬럼 목록
        categorical_cols: 범주형 컬럼 목록
        cyclical_cols: 순환형 컬럼 목록 (sin/cos 쌍)
        special_cols: 특수 처리 필요 컬럼 목록
        plant_output_cols: 발전량 관련 컬럼 목록
    """
    # 발전량 관련 컬럼 (발전소별 정규화 필요)
    plant_output_cols = ['총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', 
                          '최대(시간별_kw)', '최소(시간별_kw)', 'value']
    plant_output_cols = [col for col in plant_output_cols if col in columns]
    
    # 범주형 컬럼 (기본적으로 알려진 범주형 컬럼들)
    categorical_cols = ['미세먼지', '초미세먼지', 'sc_14', 'sc_20']
    categorical_cols = [col for col in categorical_cols if col in columns]
    
    # 순환형 컬럼 (이미 sin/cos 변환된 컬럼들)
    cyclical_pairs = [
        ('time_hour_sin', 'time_hour_cos'),
        ('time_day_sin', 'time_day_cos'),
        ('time_month_sin', 'time_month_cos'),
        ('wd_sin', 'wd_cos'),
        ('wd_14_sin', 'wd_14_cos'),
        ('wd_20_sin', 'wd_20_cos')
    ]
    
    cyclical_cols = []
    for sin_col, cos_col in cyclical_pairs:
        if sin_col in columns and cos_col in columns:
            cyclical_cols.extend([sin_col, cos_col])
    
    # 특수 처리 필요 컬럼
    special_cols = ['time_is_daylight']  # 0/1 값을 가지는 특수 컬럼
    special_cols = [col for col in special_cols if col in columns]
    
    # 나머지는 수치형으로 처리
    numerical_cols = [col for col in columns if col not in categorical_cols + 
                     cyclical_cols + special_cols + plant_output_cols]
    
    return numerical_cols, categorical_cols, cyclical_cols, special_cols, plant_output_cols

def create_preprocessor(numerical_cols, categorical_cols, cyclical_cols, special_cols, plant_output_cols):
    """
    전처리 파이프라인 생성
    
    Args:
        numerical_cols: 수치형 컬럼 목록
        categorical_cols: 범주형 컬럼 목록
        cyclical_cols: 순환형 컬럼 목록
        special_cols: 특수 처리 컬럼 목록
    
    Returns:
        preprocessor: 전처리 파이프라인
    """
    # 수치형 변수 전처리 (StandardScaler)
    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])
    
    # 범주형 변수 전처리 (LabelEncoder로 변경)
    # LabelEncoder는 각 컬럼별로 독립적으로 적용해야 하므로 'passthrough'로 설정
    categorical_transformer = 'passthrough'
    
    # 순환형 변수 전처리 (MinMaxScaler->아님! 안해도 이미 -1,1 사이임)
    '''
    cyclical_transformer = Pipeline(steps=[
        ('minmax', MinMaxScaler(feature_range=(-1, 1)))
    ])
    '''
    cyclical_transformer = 'passthrough'
    
    # 특수 변수 전처리 (그대로 유지)
    special_transformer = 'passthrough'

    # 발전량 관련 변수 전처리 (StandardScaler)
    plant_output_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())  # 발전량 값이 크므로 StandardScaler 적용
    ])    
    
    # 전처리 파이프라인 구성
    transformers = []
    
    if numerical_cols:
        transformers.append(('num', numerical_transformer, numerical_cols))
    
    if categorical_cols:
        transformers.append(('cat', categorical_transformer, categorical_cols))
    
    if cyclical_cols:
        transformers.append(('cyc', cyclical_transformer, cyclical_cols))
    
    if special_cols:
        transformers.append(('spe', special_transformer, special_cols))

    if plant_output_cols:
        transformers.append(('out', plant_output_transformer, plant_output_cols))        
    
    preprocessor = ColumnTransformer(
        transformers=transformers,
        remainder='drop'  # 지정되지 않은 컬럼 제외
    )
    
    return preprocessor

def get_feature_names(preprocessor, numerical_cols, categorical_cols, cyclical_cols, 
                      special_cols, plant_output_cols, additional_features=None):
    """
    전처리 후 특성 이름 추출
    
    Args:
        preprocessor: 학습된 전처리 파이프라인
        numerical_cols: 수치형 컬럼 목록
        categorical_cols: 범주형 컬럼 목록
        cyclical_cols: 순환형 컬럼 목록
        special_cols: 특수 처리 컬럼 목록
        plant_output_cols: 발전량 관련 컬럼 목록
        additional_features: 추가된 파생 피처 목록
    
    Returns:
        feature_names: 전처리 후 특성 이름 목록
    """
    feature_names = []
    
    # 수치형 변수 이름 (변경 없음)
    if numerical_cols:
        feature_names.extend(numerical_cols)
    
    # 범주형 변수 이름 (LabelEncoder 적용 시 이름 변경 없음)
    if categorical_cols:
        feature_names.extend(categorical_cols)
    
    # 순환형 변수 이름 (변경 없음)
    if cyclical_cols:
        feature_names.extend(cyclical_cols)
    
    # 특수 변수 이름 (변경 없음)
    if special_cols:
        feature_names.extend(special_cols)
    
    # 발전량 관련 변수 이름 (정규화 후에도 이름 유지)
    if plant_output_cols:
        feature_names.extend(plant_output_cols)
    
    # 추가 파생 피처 이름
    if additional_features:
        feature_names.extend(additional_features)
    
    return feature_names

'''def preprocess_meta_data(meta_data):
    """
    메타데이터 전처리
    
    Args:
        meta_data: 메타데이터 배열 (X_meta)
    
    Returns:
        processed_meta: 전처리된 메타데이터
        meta_feature_names: 메타데이터 피처 이름
        region_encoder: 지역 인코더 (있는 경우)
    """
    # 메타데이터가 없으면 빈 배열 반환
    if meta_data is None or len(meta_data) == 0:
        return np.array([]), [], None
    
    print("메타데이터 전처리 중...")
    
    # 메타데이터를 DataFrame으로 변환
    if isinstance(meta_data[0], dict):
        # 딕셔너리 형태인 경우
        meta_df = pd.DataFrame(meta_data)
    else:
        # 배열 형태인 경우
        meta_df = pd.DataFrame(meta_data, columns=['capacity_kw', 'plant_age', 'region'])
    
    # 메타데이터 컬럼 추출
    numeric_cols = ['capacity_kw', 'plant_age']
    numeric_cols = [col for col in numeric_cols if col in meta_df.columns]
    
    categorical_cols = ['region']
    categorical_cols = [col for col in categorical_cols if col in meta_df.columns]
    
    # 수치형 메타데이터 스케일링
    meta_scaler = StandardScaler()
    if numeric_cols:
        # NaN 값 처리
        meta_df[numeric_cols] = meta_df[numeric_cols].fillna(0)
        meta_df[numeric_cols] = meta_scaler.fit_transform(meta_df[numeric_cols])
    
    # 범주형 메타데이터 원-핫 인코딩
    region_encoder = None
    if categorical_cols:
        # NaN 값 처리
        meta_df[categorical_cols] = meta_df[categorical_cols].fillna('Unknown')
        
        # 원-핫 인코딩
        region_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        region_encoded = region_encoder.fit_transform(meta_df[categorical_cols])
        
        # 원-핫 인코딩 결과를 DataFrame으로 변환
        region_cols = [f"region_{cat}" for cat in region_encoder.categories_[0]]
        region_df = pd.DataFrame(region_encoded, columns=region_cols)
        
        # 원본 데이터프레임에서 범주형 컬럼 제거
        meta_df = meta_df.drop(columns=categorical_cols)
        
        # 원-핫 인코딩 결과 병합
        meta_df = pd.concat([meta_df, region_df], axis=1)
    
    # 메타데이터 피처 이름
    meta_feature_names = list(meta_df.columns)
    
    # 전처리된 메타데이터 반환
    return meta_df.values, meta_feature_names, region_encoder
'''


def create_region_encoder_from_all_splits(scenario_dir, split_names):
    """
    모든 split의 지역 정보를 합쳐서 OneHotEncoder 생성
    
    Args:
        scenario_dir: 시나리오 디렉토리 경로
        split_names: 분할 데이터 이름 목록
    
    Returns:
        region_encoder: 모든 지역을 포함한 OneHotEncoder
        all_regions: 발견된 모든 지역 목록
    """
    all_regions = set()
    
    # 모든 split에서 지역 정보 수집
    for split_name in split_names:
        split_dir = os.path.join(scenario_dir, split_name)
        try:
            X_meta = np.load(os.path.join(split_dir, 'X_meta.npy'), allow_pickle=True)
            
            # 지역 정보 추출 (인덱스 2가 region)
            for meta in X_meta:
                if len(meta) > 2:
                    region = str(meta[2]) if meta[2] is not None else "Unknown"
                    all_regions.add(region)
                    
        except Exception as e:
            print(f"  {split_name} 세트에서 지역 정보 수집 실패: {e}")
    
    print(f"발견된 모든 지역: {sorted(all_regions)}")
    
    # OneHotEncoder 생성
    if all_regions:
        # Unknown 지역도 포함
        if "Unknown" not in all_regions:
            all_regions.add("Unknown")
        
        # 지역 목록을 정렬하여 일관성 유지
        region_list = sorted(list(all_regions))
        
        # OneHotEncoder 학습
        region_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        region_encoder.fit(np.array(region_list).reshape(-1, 1))
        
        return region_encoder, region_list
    else:
        return None, []


def preprocess_meta_data_train_only(meta_data, region_encoder):
    """
    Train 메타데이터 전처리 (수치형만 학습, 지역은 미리 만든 encoder 사용)
    
    Args:
        meta_data: Train 메타데이터 배열
        region_encoder: 미리 생성된 지역 인코더
    
    Returns:
        processed_meta: 전처리된 메타데이터
        meta_feature_names: 메타데이터 피처 이름
        numeric_scaler: 수치형 데이터 스케일러
    """
    # 메타데이터가 없으면 빈 배열 반환
    if meta_data is None or len(meta_data) == 0:
        return np.array([]), [], None
    
    print("Train 메타데이터 전처리 중...")
    
    # 메타데이터를 DataFrame으로 변환
    if isinstance(meta_data[0], dict):
        meta_df = pd.DataFrame(meta_data)
    else:
        meta_df = pd.DataFrame(meta_data, columns=['capacity_kw', 'plant_age', 'region'])
    
    # 메타데이터 컬럼 추출
    numeric_cols = ['capacity_kw', 'plant_age']
    numeric_cols = [col for col in numeric_cols if col in meta_df.columns]
    
    categorical_cols = ['region']
    categorical_cols = [col for col in categorical_cols if col in meta_df.columns]
    
    # ✅ 수치형 메타데이터 스케일링 (Train 데이터로만 학습)
    numeric_scaler = None
    if numeric_cols:
        # NaN 값 처리
        meta_df[numeric_cols] = meta_df[numeric_cols].fillna(0)
        
        # Train 데이터로 scaler 학습
        numeric_scaler = StandardScaler()
        meta_df[numeric_cols] = numeric_scaler.fit_transform(meta_df[numeric_cols])
    
    # ✅ 범주형 메타데이터 원-핫 인코딩 (미리 만든 encoder 사용)
    if categorical_cols and region_encoder is not None:
        # NaN 값 처리
        meta_df[categorical_cols] = meta_df[categorical_cols].fillna('Unknown')
        
        # 미리 학습된 encoder로 변환
        region_encoded = region_encoder.transform(meta_df[categorical_cols])
        
        # 원-핫 인코딩 결과를 DataFrame으로 변환
        region_cols = [f"region_{cat}" for cat in region_encoder.categories_[0]]
        region_df = pd.DataFrame(region_encoded, columns=region_cols)
        
        # 원본 데이터프레임에서 범주형 컬럼 제거
        meta_df = meta_df.drop(columns=categorical_cols)
        
        # 원-핫 인코딩 결과 병합
        meta_df = pd.concat([meta_df, region_df], axis=1)
    
    # 메타데이터 피처 이름
    meta_feature_names = list(meta_df.columns)
    
    # 전처리된 메타데이터와 수치형 스케일러 반환
    return meta_df.values, meta_feature_names, numeric_scaler


def apply_meta_preprocessing(X_meta, meta_cols, region_encoder, numeric_scaler):
    """
    학습된 전처리기를 사용하여 메타데이터 변환
    
    Args:
        X_meta: 변환할 메타데이터
        meta_cols: 메타데이터 컬럼명
        region_encoder: 학습된 지역 인코더
        numeric_scaler: 학습된 수치형 스케일러
    
    Returns:
        X_meta_normalized: 전처리된 메타데이터
    """
    if X_meta is None:
        return None
    
    # 메타데이터를 DataFrame으로 변환
    if isinstance(X_meta[0], (list, np.ndarray)):
        meta_df = pd.DataFrame(X_meta, columns=meta_cols[:len(X_meta[0])])
    else:
        meta_df = pd.DataFrame(X_meta.reshape(-1, 1), columns=meta_cols[:1])
    
    # 수치형 메타데이터 변환
    numeric_meta_cols = [col for col in ['capacity_kw', 'plant_age'] if col in meta_df.columns]
    if numeric_meta_cols and numeric_scaler is not None:
        # ✅ 학습된 scaler로 transform만 수행
        meta_df[numeric_meta_cols] = meta_df[numeric_meta_cols].fillna(0)
        meta_df[numeric_meta_cols] = numeric_scaler.transform(meta_df[numeric_meta_cols])
    
    # 범주형 메타데이터 변환
    categorical_meta_cols = [col for col in ['region'] if col in meta_df.columns]
    if categorical_meta_cols and region_encoder is not None:
        # NaN 값 처리
        meta_df[categorical_meta_cols] = meta_df[categorical_meta_cols].fillna('Unknown')
        
        # ✅ 학습된 encoder로 transform만 수행
        region_encoded = region_encoder.transform(meta_df[categorical_meta_cols])
        
        # 원-핫 인코딩 결과를 DataFrame으로 변환
        region_cols = [f"region_{cat}" for cat in region_encoder.categories_[0]]
        region_df = pd.DataFrame(region_encoded, columns=region_cols)
        
        # 원본 데이터프레임에서 범주형 컬럼 제거
        meta_df = meta_df.drop(columns=categorical_meta_cols)
        
        # 원-핫 인코딩 결과 병합
        meta_df = pd.concat([meta_df, region_df], axis=1)
    
    return meta_df.values

def normalize_plant_output(data, plant_meta, plant_id, plant_output_cols):
    """
    발전량 데이터 정규화 (설비 용량 기반)
    
    Args:
        data: 발전소 데이터 (DataFrame)
        plant_meta: 발전소 메타데이터 (Dictionary)
        plant_id: 발전소 ID
        plant_output_cols: 발전량 관련 컬럼 목록
    
    Returns:
        normalized_data: 정규화된 데이터 (DataFrame)
        added_features: 추가된 파생 피처 목록
    """
    # 데이터 복사본 생성
    normalized_data = data.copy()
    added_features = []
    
    # 발전소 용량 정보가 없으면 통계 기반 정규화 수행
    if plant_meta is None or 'capacity_kw' not in plant_meta or plant_meta['capacity_kw'] == 0:
        # 각 발전량 컬럼별로 평균/표준편차 정규화 수행
        for col in plant_output_cols:
            if col in normalized_data.columns:
                # 문자열인 경우 숫자로 변환 시도
                try:
                    normalized_data[col] = pd.to_numeric(normalized_data[col], errors='coerce').astype(np.float32)
                except:
                    print(f"    경고: '{col}' 컬럼 숫자 변환 실패, 정규화 건너뜀")
                    continue
                
                # NaN 값 처리
                if normalized_data[col].isna().any():
                    print(f"    경고: '{col}' 컬럼에 NaN 값 존재, 0으로 대체")
                    normalized_data[col] = normalized_data[col].fillna(0)
                
                mean = normalized_data[col].mean()
                std = normalized_data[col].std()
                if std == 0:
                    std = 1.0  # 0으로 나누기 방지
                normalized_data[col] = (normalized_data[col] - mean) / std
        
        print(f"  발전소 {plant_id}의 용량 정보 없음: 통계 기반 정규화 수행")
        return normalized_data, added_features
    
    # 발전소 용량 정보가 있는 경우 용량 기반 정규화 수행
    capacity_kw = plant_meta['capacity_kw']
    #print(f"  발전소 {plant_id}의 용량: {capacity_kw} kW")
    
    # 발전량 데이터를 설비 용량으로 나누어 정규화 (이용률로 변환)
    for col in plant_output_cols:
        if col in normalized_data.columns:
            # 문자열인 경우 숫자로 변환 시도
            try:
                normalized_data[col] = pd.to_numeric(normalized_data[col], errors='coerce').astype(np.float32)
            except:
                print(f"    경고: '{col}' 컬럼 숫자 변환 실패, 정규화 건너뜀")
                continue
            
            # NaN 값 처리
            if normalized_data[col].isna().any():
                print(f"    경고: '{col}' 컬럼에 NaN 값 존재, 0으로 대체")
                normalized_data[col] = normalized_data[col].fillna(0)
            
            # capacity_kw가 문자열인 경우 숫자로 변환
            if isinstance(capacity_kw, str):
                try:
                    capacity_kw = float(capacity_kw)
                except:
                    print(f"    경고: 용량 값 '{capacity_kw}' 숫자 변환 실패, 정규화 건너뜀")
                    continue
            
            # 용량이 0인 경우 처리
            if capacity_kw == 0:
                print(f"    경고: 용량이 0, 정규화 대신 0으로 설정")
                normalized_data[f"{col}_ratio"] = 0
            else:
                # 발전량을 용량으로 나누어 이용률로 변환 (0~1 범위)
                normalized_data[f"{col}_ratio"] = normalized_data[col] / capacity_kw
            
            added_features.append(f"{col}_ratio")
    
    # 하루 전 같은 시간대 대비 변화율 계산 (value 컬럼 대상)
    if 'value' in normalized_data.columns and len(normalized_data) > 24:
        # 문자열인 경우 숫자로 변환 시도
        if not pd.api.types.is_numeric_dtype(normalized_data['value']):
            try:
                normalized_data['value'] = pd.to_numeric(normalized_data['value'], errors='coerce')
                normalized_data['value'] = normalized_data['value'].fillna(0)
            except:
                print(f"    경고: 'value' 컬럼 숫자 변환 실패, 변화율 계산 건너뜀")
                return normalized_data, added_features
        
        # 24시간(하루) 전 발전량
        prev_day_values = normalized_data['value'].shift(24)
        
        # 변화율 계산 (이전 값이 0인 경우 처리)
        normalized_data['value_day_change'] = normalized_data['value'].div(
            prev_day_values.replace(0, 1e-8)).replace([np.inf, -np.inf], 0) - 1
        
        # NaN 값을 0으로 대체
        normalized_data['value_day_change'] = normalized_data['value_day_change'].fillna(0)
        
        # 이상치 제한 (-1 ~ 1 범위로 클리핑)
        normalized_data['value_day_change'] = np.clip(normalized_data['value_day_change'], -1, 1)
        
        added_features.append('value_day_change')
    
    return normalized_data, added_features


def normalize_single_target_value(y_sample, plant_id, plant_meta):
    """
    단일 샘플의 Y값 정규화 (X와 동일한 방식)
    """
    # 용량 기반 정규화
    capacity_kw = plant_meta['capacity_kw']
    if isinstance(capacity_kw, str):
        try:
            capacity_kw = float(capacity_kw)
        except:
            capacity_kw = 0
    
    if capacity_kw > 0:
        return y_sample / capacity_kw, 'capacity_based', capacity_kw

    # 통계 기반 정규화는 개별 샘플로는 불가능하므로 원본 반환
    return y_sample, 'no_normalization', None    


def normalize_data(input_dir, output_dir, scenario, split_names=['train', 'valid', 'test', 'external_test']):
    """
    데이터 정규화 및 저장
    
    Args:
        input_dir: 입력 데이터 디렉토리
        output_dir: 출력 데이터 디렉토리
        scenario: 시나리오 이름 ('14시' 또는 '20시')
        split_names: 분할 데이터 이름 목록
    """
    print(f"===== {scenario} 시나리오 데이터 정규화 =====")
    
    # 컬럼 정보 로드
    column_info = load_column_info(input_dir, scenario)
    historical_cols = column_info['historical_cols']
    forecast_cols = column_info['forecast_cols']
    meta_cols = column_info.get('meta_cols', [])
    
    # 특성 유형 식별
    historical_numerical, historical_categorical, historical_cyclical, historical_special, historical_plant_output = identify_column_types(historical_cols)
    forecast_numerical, forecast_categorical, forecast_cyclical, forecast_special, forecast_plant_output = identify_column_types(forecast_cols)
    
    print(f"Historical columns ({len(historical_cols)}):")
    print(f"  Numerical: {len(historical_numerical)}")
    print(f"  Categorical: {len(historical_categorical)}")
    print(f"  Cyclical: {len(historical_cyclical)}")
    print(f"  Special: {len(historical_special)}")
    print(f"  Plant Output: {len(historical_plant_output)}")
    
    print(f"Forecast columns ({len(forecast_cols)}):")
    print(f"  Numerical: {len(forecast_numerical)}")
    print(f"  Categorical: {len(forecast_categorical)}")
    print(f"  Cyclical: {len(forecast_cyclical)}")
    print(f"  Special: {len(forecast_special)}")
    print(f"  Plant Output: {len(forecast_plant_output)}")
    
    print(f"Meta columns ({len(meta_cols)}): {meta_cols}")
    
    # 시나리오 디렉토리 생성
    scenario_dir = os.path.join(input_dir, scenario)
    output_scenario_dir = os.path.join(output_dir, scenario)
    os.makedirs(output_scenario_dir, exist_ok=True)
    
    # 전처리 파이프라인 생성 (Historical 데이터용)
    historical_preprocessor = create_preprocessor(
        historical_numerical, 
        historical_categorical, 
        historical_cyclical, 
        historical_special,
        historical_plant_output
    )
    
    # 전처리 파이프라인 생성 (Forecast 데이터용)
    forecast_preprocessor = create_preprocessor(
        forecast_numerical, 
        forecast_categorical, 
        forecast_cyclical, 
        forecast_special,
        forecast_plant_output
    )
    
    # 레이블 인코더 생성 (범주형 변수용)
    historical_label_encoders = {}
    forecast_label_encoders = {}

    # ✅ 1단계: 모든 split에서 지역 정보 수집하여 OneHotEncoder 생성
    print("\n모든 split에서 지역 정보 수집 중...")
    region_encoder, all_regions = create_region_encoder_from_all_splits(scenario_dir, split_names)
    
    
    # 학습 데이터 로드
    train_dir = os.path.join(scenario_dir, 'train')
    X_historical_train = np.load(os.path.join(train_dir, 'X_historical.npy'), allow_pickle=True)
    X_forecast_train = np.load(os.path.join(train_dir, 'X_forecast.npy'), allow_pickle=True)
    y_train = np.load(os.path.join(train_dir, 'y.npy'), allow_pickle=True)
    plants_train = np.load(os.path.join(train_dir, 'plants.npy'), allow_pickle=True)
    
    # 메타데이터 로드 (있는 경우)
    X_meta_train = None
    try:
        X_meta_train = np.load(os.path.join(train_dir, 'X_meta.npy'), allow_pickle=True)
        print("메타데이터 로드 성공")
    except:
        print("메타데이터 파일이 없거나 로드할 수 없습니다.")

    # ✅ 2단계: Train 데이터로 수치형 스케일러만 학습 (지역은 미리 만든 encoder 사용)
    processed_meta, meta_feature_names, numeric_scaler = preprocess_meta_data_train_only(
        X_meta_train, region_encoder
    )        
    
    # 발전소별 메타데이터 사전 구성
    # 발전소별 메타데이터 사전 구성 - 모든 데이터 세트에서 수집
    plant_meta_dict = {}

    # 모든 분할 데이터 세트에서 메타데이터 수집
    for split_name in ['train', 'valid', 'test', 'external_test']:
        split_dir = os.path.join(scenario_dir, split_name)
        try:
            # 해당 세트의 발전소 및 메타데이터 로드
            plants_split = np.load(os.path.join(split_dir, 'plants.npy'), allow_pickle=True)
            X_meta_split = np.load(os.path.join(split_dir, 'X_meta.npy'), allow_pickle=True)
            
            # 메타데이터 사전에 추가
            unique_plants = np.unique(plants_split)
            for plant_id in unique_plants:
                if plant_id not in plant_meta_dict:  # 아직 사전에 없는 발전소만 추가
                    # 현재 발전소의 첫 번째 샘플 인덱스 찾기
                    plant_idx = np.where(plants_split == plant_id)[0][0]
                    
                    # 해당 발전소의 메타데이터 저장
                    plant_meta_dict[plant_id] = {
                        'capacity_kw': X_meta_split[plant_idx][0] if len(X_meta_split[plant_idx]) > 0 else 0,
                        'plant_age': X_meta_split[plant_idx][1] if len(X_meta_split[plant_idx]) > 1 else 0,
                        'region': X_meta_split[plant_idx][2] if len(X_meta_split[plant_idx]) > 2 else "Unknown"
                    }
        except:
            print(f"  {split_name} 세트에서 메타데이터를 로드할 수 없습니다.")

    print(f"발전소 메타데이터 사전 구성 완료: {len(plant_meta_dict)}개 발전소")
    
    # 데이터 형태 확인
    print(f"Train data shapes:")
    print(f"  X_historical_train: {X_historical_train.shape}")
    print(f"  X_forecast_train: {X_forecast_train.shape}")
    print(f"  y_train: {y_train.shape}")
    if X_meta_train is not None:
        print(f"  X_meta_train: {X_meta_train.shape}")
    
    # 시나리오에 따른 패딩 식별
    cutoff_hour = 14 if scenario == '14시' else 20
    padding_size = 24 - cutoff_hour
    
    # 3D 데이터를 2D로 변환 (시간 차원 펼치기)
    n_samples_train, n_timesteps_hist, n_features_hist = X_historical_train.shape
    n_samples_train, n_timesteps_forecast, n_features_forecast = X_forecast_train.shape
    
    # 패딩 마스크 생성 (패딩이 아닌 실제 데이터 위치: True)
    # 형태: [n_samples, n_timesteps]
    mask_hist = np.ones((n_samples_train, n_timesteps_hist), dtype=bool)
    
    # history_days를 추정 (전체 시간 단계에서 하루 24시간을 뺀 값)
    history_days = (n_timesteps_hist - 24) // 24
    
    # 패딩 위치 계산: D-1일의 cutoff_hour 이후 시간
    # 패딩 시작 인덱스: history_days*24 + cutoff_hour
    padding_start_idx = history_days * 24 + cutoff_hour
    
    # 패딩 위치 마스킹
    mask_hist[:, padding_start_idx:] = False
    
    # 마스크를 2D로 변환
    mask_hist_2d = mask_hist.reshape(-1)
    
    # 실제 데이터만 추출하여 2D로 변환
    X_historical_train_2d = X_historical_train.reshape(-1, n_features_hist)
    X_forecast_train_2d = X_forecast_train.reshape(-1, n_features_forecast)
    
    # 패딩을 제외한 실제 데이터만 선택
    X_historical_train_real = X_historical_train_2d[mask_hist_2d]
    
    # 2D 마스크에 대응하는 샘플 인덱스
    sample_indices_2d = np.repeat(np.arange(n_samples_train), n_timesteps_hist)[mask_hist_2d]
    
    # 2D 마스크에 대응하는 발전소 이름
    plants_2d = plants_train[sample_indices_2d]
    
    # 데이터프레임으로 변환
    historical_df_train = pd.DataFrame(X_historical_train_real, columns=historical_cols)
    forecast_df_train = pd.DataFrame(X_forecast_train_2d, columns=forecast_cols)
    
    # 범주형 변수에 대한 LabelEncoder 학습 및 변환 매핑 저장
    label_mapping = {}
            
    # Historical 데이터의 범주형 변수 처리
    for col in historical_categorical:
        print(f"\n범주형 특성 '{col}' 레이블 인코딩:")
        
        if col == '미세먼지' or col == '초미세먼지':
            # 미세먼지/초미세먼지 순서 정의 (좋음 > 보통 > 나쁨 > 매우나쁨)
            ordered_categories = ['좋음', '보통', '나쁨', '매우나쁨']
            
            # 데이터에 있는 추가 카테고리 확인 (예: nan)
            data_categories = set(historical_df_train[col].astype(str).unique())
            for category in data_categories:
                if category not in ordered_categories and category != 'nan':
                    ordered_categories.append(category)
            
            # nan 값이 있으면 마지막에 추가
            if 'nan' in data_categories:
                ordered_categories.append('nan')
            
            # 직접 매핑 생성
            mapping = {cat: idx for idx, cat in enumerate(ordered_categories)}
            
            # 직접 변환 함수 생성
            def transform_func(x, mapping=mapping):
                return np.array([mapping.get(str(val), len(mapping) - 1) for val in x])
            
            # 변환 함수 저장
            historical_label_encoders[col] = transform_func
            
        elif col == 'sc_14' or col == 'sc_20':
            # 하늘상태(sc) 순서 정의 (1=맑음, 2=구름조금, 3=구름많음, 4=흐림)
            ordered_categories = ['1.0', '2.0', '3.0', '4.0']
            
            # 데이터에 있는 추가 카테고리 확인
            data_categories = set(historical_df_train[col].astype(str).unique())
            for category in data_categories:
                if category not in ordered_categories and category != 'nan':
                    ordered_categories.append(category)
            
            # nan 값이 있으면 마지막에 추가
            if 'nan' in data_categories:
                ordered_categories.append('nan')
            
            # 직접 매핑 생성
            mapping = {cat: idx for idx, cat in enumerate(ordered_categories)}
            
            # 직접 변환 함수 생성
            def transform_func(x, mapping=mapping):
                return np.array([mapping.get(str(val), len(mapping) - 1) for val in x])
            
            # 변환 함수 저장
            historical_label_encoders[col] = transform_func
            
        else:
            # 기존 방식 사용 (다른 범주형 변수)
            encoder = LabelEncoder()
            values = historical_df_train[col].astype(str).unique()
            encoder.fit(values)
            historical_label_encoders[col] = encoder
            
            # 기존 매핑 정보 저장
            mapping = {val: int(idx) for idx, val in enumerate(encoder.classes_)}
        
        # 매핑 정보 저장 및 출력
        label_mapping[col] = mapping
        print(f"  원본 카테고리: {list(mapping.keys())}")
        print(f"  변환 매핑: {mapping}")

    # Forecast 데이터의 범주형 변수 처리
    for col in forecast_categorical:
        # 이미 처리된 컬럼은 건너뛰기
        if col in historical_label_encoders:
            forecast_label_encoders[col] = historical_label_encoders[col]
            continue
        
        print(f"\n범주형 특성 '{col}' 레이블 인코딩:")
        
        if col == '미세먼지' or col == '초미세먼지':
            # 미세먼지/초미세먼지 순서 정의 (좋음 > 보통 > 나쁨 > 매우나쁨)
            ordered_categories = ['좋음', '보통', '나쁨', '매우나쁨']
            
            # 데이터에 있는 추가 카테고리 확인 (예: nan)
            data_categories = set(forecast_df[col].astype(str).unique())
            for category in data_categories:
                if category not in ordered_categories and category != 'nan':
                    ordered_categories.append(category)
            
            # nan 값이 있으면 마지막에 추가
            if 'nan' in data_categories:
                ordered_categories.append('nan')
            
            # 직접 매핑 생성
            mapping = {cat: idx for idx, cat in enumerate(ordered_categories)}
            
            # 직접 변환 함수 생성
            def transform_func(x, mapping=mapping):
                return np.array([mapping.get(str(val), len(mapping) - 1) for val in x])
            
            # 변환 함수 저장
            forecast_label_encoders[col] = transform_func
            
        elif col == 'sc_14' or col == 'sc_20':
            # 하늘상태(sc) 순서 정의 (1=맑음, 2=구름조금, 3=구름많음, 4=흐림)
            ordered_categories = ['1.0', '2.0', '3.0', '4.0']
            
            # 데이터에 있는 추가 카테고리 확인
            data_categories = set(forecast_df[col].astype(str).unique())
            for category in data_categories:
                if category not in ordered_categories and category != 'nan':
                    ordered_categories.append(category)
            
            # nan 값이 있으면 마지막에 추가
            if 'nan' in data_categories:
                ordered_categories.append('nan')
            
            # 직접 매핑 생성
            mapping = {cat: idx for idx, cat in enumerate(ordered_categories)}
            
            # 직접 변환 함수 생성
            def transform_func(x, mapping=mapping):
                return np.array([mapping.get(str(val), len(mapping) - 1) for val in x])
            
            # 변환 함수 저장
            forecast_label_encoders[col] = transform_func
            
        else:
            # 기존 방식 사용 (다른 범주형 변수)
            encoder = LabelEncoder()
            values = forecast_df[col].astype(str).unique()
            encoder.fit(values)
            forecast_label_encoders[col] = encoder
            
            # 기존 매핑 정보 저장
            mapping = {val: int(idx) for idx, val in enumerate(encoder.classes_)}
        
        # 매핑 정보 저장 및 출력
        label_mapping[col] = mapping
        print(f"  원본 카테고리: {list(mapping.keys())}")
        print(f"  변환 매핑: {mapping}")
        
    # 발전소별 데이터 처리 및 파생 피처 추가를 위한 준비
    enhanced_historical_df_train = pd.DataFrame()
    all_added_features = []
    
    # 발전소별 데이터 정규화 및 파생 피처 추가
    print("\n발전소별 발전량 데이터 정규화 및 파생 피처 추가:")
    unique_plants = np.unique(plants_2d)
    for plant_id in unique_plants:
        # 현재 발전소 데이터 추출
        plant_mask = plants_2d == plant_id
        plant_data = historical_df_train[plant_mask].copy()
        
        # 발전소 메타데이터 가져오기
        plant_meta = plant_meta_dict.get(plant_id, None)
        
        # 발전량 데이터 정규화 및 파생 피처 추가
        normalized_plant_data, added_features = normalize_plant_output(
            plant_data, plant_meta, plant_id, historical_plant_output
        )
        
        # 모든 파생 피처 목록 업데이트
        for feature in added_features:
            if feature not in all_added_features:
                all_added_features.append(feature)
        
        # 결과 합치기
        enhanced_historical_df_train = pd.concat([enhanced_historical_df_train, normalized_plant_data])
    
    # 데이터 정렬 (원래 순서로)
    enhanced_historical_df_train = enhanced_historical_df_train.sort_index()
    
    
    # 전처리 파이프라인 학습 (패딩 제외한 실제 데이터로만)
    print("\nHistorical preprocessor fitting (excluding padding)...")
    historical_preprocessor.fit(enhanced_historical_df_train[[col for col in enhanced_historical_df_train.columns 
                                                             if col not in all_added_features]])
    
    print("Forecast preprocessor fitting...")
    forecast_preprocessor.fit(forecast_df_train)
    
    # 정규화된 특성 이름 가져오기
    historical_feature_names = get_feature_names(
        historical_preprocessor, 
        historical_numerical, 
        historical_categorical, 
        historical_cyclical, 
        historical_special,
        historical_plant_output,
        all_added_features
    )
    
    forecast_feature_names = get_feature_names(
        forecast_preprocessor, 
        forecast_numerical, 
        forecast_categorical, 
        forecast_cyclical, 
        forecast_special,
        forecast_plant_output
    )
    
    # 변환된 특성 정보 저장
    feature_info = {
        'historical_feature_names': historical_feature_names,
        'forecast_feature_names': forecast_feature_names,
        'meta_feature_names': meta_feature_names,
        'historical_cols': historical_cols,
        'forecast_cols': forecast_cols,
        'meta_cols': meta_cols,
        'historical_numerical': historical_numerical,
        'historical_categorical': historical_categorical,
        'historical_cyclical': historical_cyclical,
        'historical_special': historical_special,
        'historical_plant_output': historical_plant_output,
        'forecast_numerical': forecast_numerical,
        'forecast_categorical': forecast_categorical,
        'forecast_cyclical': forecast_cyclical,
        'forecast_special': forecast_special,
        'forecast_plant_output': forecast_plant_output,
        'added_features': all_added_features,
        'label_mapping': label_mapping,
        'padding_info': {
                    'cutoff_hour': cutoff_hour,
                    'padding_size': padding_size,
                    'padding_start_idx': padding_start_idx,
                    'history_days': history_days
                }
    }

    # 컬럼 순서 정보 추가
    column_order_info = {
        'historical_feature_order': historical_feature_names,
        'forecast_feature_order': forecast_feature_names,
        'meta_feature_order': meta_feature_names
    }

    # 기존 feature_info 딕셔너리에 column_order_info 추가
    feature_info.update(column_order_info)    
            
    # 변환된 특성 정보를 JSON 파일로 저장
    with open(os.path.join(output_scenario_dir, 'feature_info.json'), 'w') as f:
        json.dump(feature_info, f, indent=2)
    
    print(f"변환된 특성 수:")
    print(f"  Historical: {len(historical_feature_names)}")
    print(f"  Forecast: {len(forecast_feature_names)}")
    print(f"  Meta: {len(meta_feature_names)}")
    
    # 각 분할 데이터 세트 처리
    for split_name in split_names:
        print(f"\nProcessing {split_name} data...")
        
        # 데이터 로드
        split_dir = os.path.join(scenario_dir, split_name)
        X_historical = np.load(os.path.join(split_dir, 'X_historical.npy'), allow_pickle=True)
        X_forecast = np.load(os.path.join(split_dir, 'X_forecast.npy'), allow_pickle=True)
        y = np.load(os.path.join(split_dir, 'y.npy'), allow_pickle=True)
        dates = np.load(os.path.join(split_dir, 'dates.npy'), allow_pickle=True)
        plants = np.load(os.path.join(split_dir, 'plants.npy'), allow_pickle=True)
        
        # 메타데이터 로드 (있는 경우)
        X_meta = None
        try:
            X_meta = np.load(os.path.join(split_dir, 'X_meta.npy'), allow_pickle=True)
        except:
            print(f"  {split_name} 세트에 대한 메타데이터 파일 없음")
        
        # 데이터 형태 확인
        print(f"  {split_name} shapes:")
        print(f"    X_historical: {X_historical.shape}")
        print(f"    X_forecast: {X_forecast.shape}")
        print(f"    y: {y.shape}")
        if X_meta is not None:
            print(f"    X_meta: {X_meta.shape}")
        
        # 3D 데이터를 2D로 변환 (시간 차원 펼치기)
        n_samples, n_timesteps_hist, n_features_hist = X_historical.shape
        n_samples, n_timesteps_forecast, n_features_forecast = X_forecast.shape
        
        # 패딩 마스크 생성
        mask_hist = np.ones((n_samples, n_timesteps_hist), dtype=bool)
        mask_hist[:, padding_start_idx:] = False
        
        # 결과 저장을 위한 정규화된 데이터 배열 초기화
        # 기존 특성 + 추가된 파생 피처 수를 고려한 크기
        historical_output_dim = len(historical_feature_names)
        X_historical_normalized = np.full((n_samples, n_timesteps_hist, historical_output_dim), -1.0, dtype=np.float32)
        
        # 각 샘플별로 처리
        for i in range(n_samples):
            # 현재 발전소 이름
            plant_id = plants[i]
            
            # 현재 발전소 메타데이터
            plant_meta = plant_meta_dict.get(plant_id, None)
            
            # 현재 샘플에서 패딩이 아닌 실제 데이터 추출
            real_data_indices = np.where(mask_hist[i])[0]
            real_data = X_historical[i, real_data_indices]
            
            # 실제 데이터를 DataFrame으로 변환
            real_df = pd.DataFrame(real_data, columns=historical_cols)
            
            # 범주형 변수 레이블 인코딩 적용
            for col in historical_categorical:
                # NaN 값 처리 (문자열로 변환)
                real_df[col] = real_df[col].astype(str)
                
                # 레이블 인코더 적용
                encoder = historical_label_encoders[col]
                
                if isinstance(encoder, LabelEncoder):
                    # 기존 LabelEncoder 방식
                    # 알 수 없는 범주 처리 (학습 데이터에 없는 값)
                    for val in real_df[col].unique():
                        if val not in encoder.classes_:
                            print(f"    경고: '{col}'에서 학습 데이터에 없는 값 '{val}' 발견됨. '0'으로 대체")
                            real_df.loc[real_df[col] == val, col] = encoder.classes_[0]
                    
                    real_df[col] = encoder.transform(real_df[col])
                else:
                    # 직접 생성한 변환 함수 사용
                    real_df[col] = encoder(real_df[col])
            
            # 발전량 데이터 정규화 및 파생 피처 추가
            normalized_real_df, _ = normalize_plant_output(
                real_df, plant_meta, plant_id, historical_plant_output
            )
            
            # 누락된 파생 피처 추가 (모든 발전소에 동일한 피처 세트 유지)
            for feature in all_added_features:
                if feature not in normalized_real_df.columns:
                    normalized_real_df[feature] = 0.0
            
            # 기본 피처에 대한 전처리 파이프라인 적용
            preprocessor_input = normalized_real_df[[col for col in normalized_real_df.columns 
                                                    if col not in all_added_features]]
            preprocessed_data = historical_preprocessor.transform(preprocessor_input)
            
            # 추가 파생 피처 데이터 추출
            additional_data = normalized_real_df[all_added_features].values
            
            # 전처리된 데이터와 추가 파생 피처 결합
            combined_data = np.hstack([preprocessed_data, additional_data])
            combined_data = combined_data.astype(np.float32) 
            
            # 결합된 데이터 형태 확인 및 필요시 조정
            if combined_data.shape[1] != historical_output_dim:
                print(f"    경고: 데이터 차원 불일치 - 예상: {historical_output_dim}, 실제: {combined_data.shape[1]}")
                # 필요시 차원 조정 (예: 0으로 패딩)
                if combined_data.shape[1] < historical_output_dim:
                    padding_dim = historical_output_dim - combined_data.shape[1]
                    combined_data = np.hstack([combined_data, np.zeros((combined_data.shape[0], padding_dim))])
                else:
                    combined_data = combined_data[:, :historical_output_dim]

            # 정규화된 데이터를 원래 위치에 삽입
            X_historical_normalized[i, real_data_indices] = combined_data


        
        # 예보 데이터 처리 (패딩 없음)
        X_forecast_2d = X_forecast.reshape(n_samples * n_timesteps_forecast, n_features_forecast)
        forecast_df = pd.DataFrame(X_forecast_2d, columns=forecast_cols)
        
        # 범주형 변수 레이블 인코딩 적용
        for col in forecast_categorical:
            # NaN 값 처리 (문자열로 변환)
            forecast_df[col] = forecast_df[col].astype(str)
            
            # 레이블 인코더 적용
            encoder = forecast_label_encoders[col]
            
            if isinstance(encoder, LabelEncoder):
                # 기존 LabelEncoder 방식
                # 알 수 없는 범주 처리 (학습 데이터에 없는 값)
                for val in forecast_df[col].unique():
                    if val not in encoder.classes_:
                        print(f"    경고: '{col}'에서 학습 데이터에 없는 값 '{val}' 발견됨. '0'으로 대체")
                        forecast_df.loc[forecast_df[col] == val, col] = encoder.classes_[0]
                
                forecast_df[col] = encoder.transform(forecast_df[col])
            else:
                # 직접 생성한 변환 함수 사용 (알 수 없는 값도 함수 내에서 처리됨)
                forecast_df[col] = encoder(forecast_df[col])
                
        X_forecast_transformed = forecast_preprocessor.transform(forecast_df)
        X_forecast_normalized = X_forecast_transformed.reshape(n_samples, n_timesteps_forecast, -1)
        
        # 메타데이터 정규화 (있는 경우)
        # ✅ 학습된 전처리기로 메타데이터 변환 (지역은 모든 split 기반, 수치형은 train 기반)
        X_meta_normalized = apply_meta_preprocessing(
            X_meta, meta_cols, region_encoder, numeric_scaler
        )
        
        
        # 정규화된 데이터 형태 출력
        print(f"    Normalized X_historical: {X_historical_normalized.shape}")
        print(f"    Normalized X_forecast: {X_forecast_normalized.shape}")
        if X_meta_normalized is not None:
            print(f"    Normalized X_meta: {X_meta_normalized.shape}")

        # Y값 정규화 적용 (X와 동일한 방식으로 개별 처리)
        y_normalized = np.zeros_like(y, dtype=np.float32)
        for i in range(n_samples):
            plant_id = plants[i]

            # 현재 발전소 메타 데이터
            plant_meta = plant_meta_dict.get(plant_id, None)
            y_normalized[i], _, _ = normalize_single_target_value(
                y[i], plant_id, plant_meta
            )            
        
        # 출력 디렉토리 생성
        output_split_dir = os.path.join(output_scenario_dir, split_name)
        os.makedirs(output_split_dir, exist_ok=True)
        
        # 정규화된 데이터와 패딩 마스크 저장
        np.save(os.path.join(output_split_dir, 'X_historical.npy'), X_historical_normalized)
        np.save(os.path.join(output_split_dir, 'X_forecast.npy'), X_forecast_normalized)
        np.save(os.path.join(output_split_dir, 'padding_mask.npy'), mask_hist)
        #np.save(os.path.join(output_split_dir, 'y.npy'), y)
        np.save(os.path.join(output_split_dir, 'y.npy'), y_normalized)  # 정규화된 Y 저장
        np.save(os.path.join(output_split_dir, 'y_original.npy'), y)   # 원본 Y도 저장        
        np.save(os.path.join(output_split_dir, 'dates.npy'), dates)
        np.save(os.path.join(output_split_dir, 'plants.npy'), plants)
        
        # 메타데이터 저장 (있는 경우)
        if X_meta_normalized is not None:
            np.save(os.path.join(output_split_dir, 'X_meta.npy'), X_meta_normalized)
    
    # 전처리 파이프라인 및 인코더 저장
    import pickle
    
    # 전처리 파이프라인 저장 디렉토리 생성
    models_dir = os.path.join(output_scenario_dir, 'preprocessors')
    os.makedirs(models_dir, exist_ok=True)
    
    # 전처리 파이프라인 저장
    with open(os.path.join(models_dir, 'historical_preprocessor.pkl'), 'wb') as f:
        pickle.dump(historical_preprocessor, f)
    
    with open(os.path.join(models_dir, 'forecast_preprocessor.pkl'), 'wb') as f:
        pickle.dump(forecast_preprocessor, f)

    # 지역 인코더 저장 (모든 split 기반)
    if region_encoder is not None:
        with open(os.path.join(models_dir, 'region_encoder.pkl'), 'wb') as f:
            pickle.dump(region_encoder, f)
        print(f"지역 인코더 저장 완료 - 총 {len(all_regions)}개 지역")
    
    # 수치형 스케일러 저장 (train 기반)
    if numeric_scaler is not None:
        with open(os.path.join(models_dir, 'numeric_meta_scaler.pkl'), 'wb') as f:
            pickle.dump(numeric_scaler, f)
        print("수치형 메타데이터 스케일러 저장 완료")        
    
    # 레이블 인코더 저장
    #with open(os.path.join(models_dir, 'historical_label_encoders.pkl'), 'wb') as f:
    #    pickle.dump(historical_label_encoders, f)
    
    #with open(os.path.join(models_dir, 'forecast_label_encoders.pkl'), 'wb') as f:
    #    pickle.dump(forecast_label_encoders, f)
    
    # 지역 인코더 저장 (있는 경우)
    if region_encoder is not None:
        with open(os.path.join(models_dir, 'region_encoder.pkl'), 'wb') as f:
            pickle.dump(region_encoder, f)
    
    print(f"{scenario} 시나리오 데이터 정규화 완료!")


# 경로 설정
input_dir = '../data/forecast_modeling_data'
output_dir = '../data/normalized_forecast_modeling_data'

# 출력 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# 시나리오별 데이터 정규화
for scenario in ['14시','20시']: # '20시'
    normalize_data(input_dir, output_dir, scenario)

print("모든 데이터 정규화 완료!")

===== 14시 시나리오 데이터 정규화 =====
Historical columns (46):
  Numerical: 27
  Categorical: 3
  Cyclical: 8
  Special: 1
  Plant Output: 7
Forecast columns (5):
  Numerical: 4
  Categorical: 1
  Cyclical: 0
  Special: 0
  Plant Output: 0
Meta columns (3): ['capacity_kw', 'plant_age', 'region']

모든 split에서 지역 정보 수집 중...
발견된 모든 지역: ['강원도', '경기도', '경상남도', '부산', '인천광역시', '제주특별자치도', '충청북도']
메타데이터 로드 성공
Train 메타데이터 전처리 중...
발전소 메타데이터 사전 구성 완료: 37개 발전소
Train data shapes:
  X_historical_train: (15895, 48, 46)
  X_forecast_train: (15895, 24, 5)
  y_train: (15895, 24)
  X_meta_train: (15895, 3)


/Users/soomin/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(



범주형 특성 '미세먼지' 레이블 인코딩:
  원본 카테고리: ['좋음', '보통', '나쁨', '매우나쁨']
  변환 매핑: {'좋음': 0, '보통': 1, '나쁨': 2, '매우나쁨': 3}

범주형 특성 '초미세먼지' 레이블 인코딩:
  원본 카테고리: ['좋음', '보통', '나쁨', '매우나쁨']
  변환 매핑: {'좋음': 0, '보통': 1, '나쁨': 2, '매우나쁨': 3}

범주형 특성 'sc_14' 레이블 인코딩:
  원본 카테고리: ['1.0', '2.0', '3.0', '4.0', '0.0', 'nan']
  변환 매핑: {'1.0': 0, '2.0': 1, '3.0': 2, '4.0': 3, '0.0': 4, 'nan': 5}

발전소별 발전량 데이터 정규화 및 파생 피처 추가:

Historical preprocessor fitting (excluding padding)...
Forecast preprocessor fitting...
변환된 특성 수:
  Historical: 54
  Forecast: 5
  Meta: 10

Processing train data...
  train shapes:
    X_historical: (15895, 48, 46)
    X_forecast: (15895, 24, 5)
    y: (15895, 24)
    X_meta: (15895, 3)


/Users/soomin/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


    Normalized X_historical: (15895, 48, 54)
    Normalized X_forecast: (15895, 24, 5)
    Normalized X_meta: (15895, 10)

Processing valid data...
  valid shapes:
    X_historical: (3406, 48, 46)
    X_forecast: (3406, 24, 5)
    y: (3406, 24)
    X_meta: (3406, 3)


...

    Normalized X_historical: (3406, 48, 54)
    Normalized X_forecast: (3406, 24, 5)
    Normalized X_meta: (3406, 10)

Processing test data...
  test shapes:
    X_historical: (3423, 48, 46)
    X_forecast: (3423, 24, 5)
    y: (3423, 24)
    X_meta: (3423, 3)


/Users/soomin/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


    Normalized X_historical: (3423, 48, 54)
    Normalized X_forecast: (3423, 24, 5)
    Normalized X_meta: (3423, 10)

Processing external_test data...
  external_test shapes:
    X_historical: (2017, 48, 46)
    X_forecast: (2017, 24, 5)
    y: (2017, 24)
    X_meta: (2017, 3)


/Users/soomin/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


    Normalized X_historical: (2017, 48, 54)
    Normalized X_forecast: (2017, 24, 5)
    Normalized X_meta: (2017, 10)
지역 인코더 저장 완료 - 총 8개 지역
수치형 메타데이터 스케일러 저장 완료
20시 시나리오 데이터 정규화 완료!
모든 데이터 정규화 완료!


In [48]:
a = np.load('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/normalized_forecast_modeling_data/14시/external_test/X_historical.npy'    , allow_pickle=True )
print(a.shape)

np.set_printoptions(precision=3, suppress=True, linewidth=120)
print(a[0, 0, :])

(2017, 48, 54)
[-1.232 -0.848 -0.12  -0.476 -0.567 -1.067  1.267  1.687 -0.743 -0.514 -0.473  0.016 -0.333  0.3   -1.3    0.043  0.45
 -1.402  2.125  0.259  0.265 -0.874  1.356 -1.259 -0.835 -0.908  0.257  1.     1.     2.     0.     1.    -0.417  0.909
 -0.     1.     1.     0.     0.    -0.704 -0.704 -0.699  0.    -0.699 -0.503 -0.43   2.153  0.09   0.425  0.     0.425
  0.     0.     0.   ]


In [49]:
print(a[0, 46, :])

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


In [50]:
a = np.load('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/forecast_modeling_data/14시/external_test/X_historical.npy',allow_pickle=True)
print(a.shape)
a[0,0,:]

(2017, 48, 46)


array([637.6, 26.567, 125.7, 0.0, 125.7, 0.1, 0.0, 1.3, 48.0, 0.0, 1.2, 90.0, 3.2, 1021.3, 1030.1, 0.0, 0.0, 3.0, 3.0,
       18.0, 1962.0, -0.9, 0.004, 0.5, 0.007, 0.0411, 43.0, 24.0, '보통', '보통', 0.0, 1.0, -0.41719360261231697,
       0.9088176373395028, -2.4492935982947064e-16, 1.0, 0, 0.0, 17.25, 1.0, 6.123233995736766e-17, 2.0, 108.0, '3.0',
       1.0, 23.0], dtype=object)

In [51]:
a = np.load('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/forecast_modeling_data/14시/external_test/X_meta.npy',allow_pickle=True)
print(a.shape)
a[0,]

(2017, 3)


array(['296.09999999999997', '5.0', '인천광역시'], dtype='<U32')

In [52]:
a = np.load('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/normalized_forecast_modeling_data/14시/external_test/X_meta.npy',allow_pickle=True)
print(a.shape)
a[0,]

(2017, 10)


array([-0.727, -1.231,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ])

In [53]:
import numpy as np

a = np.load('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/normalized_forecast_modeling_data/14시/external_test/y.npy',allow_pickle=True)
print(a.shape)
a[450]

(2017, 24)


array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.019, 0.105, 0.12 , 0.131, 0.081, 0.114, 0.138,
       0.093, 0.019, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ], dtype=float32)

In [54]:
a = np.load('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/normalized_forecast_modeling_data/14시/external_test/y_original.npy',allow_pickle=True)
print(a.shape)
a[450]

(2017, 24)


array([ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.24,  2.16, 12.12, 13.8 , 15.12,  9.36, 13.08, 15.84,
       10.68,  2.16,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ])